In [1]:
import mne, glob
import mne.io
from scipy.io import loadmat
import h5py
import pandas as pd
import pymatreader
import timeit
import numpy as np
import matplotlib.pyplot as plt
#mne.sys_info()
from copy import deepcopy
import os
# import our functions
from mne_methods import *
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs, 
                               create_eog_epochs)

## Data sets

In [2]:
sim_training_data = [# A
                     [r'EEG_manual_100-25cha_0noise_10', 10, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_10', 1000, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_30', 10, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_30', 1000, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_50', 10, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_50', 1000, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_75', 10, [[], [], [], [], [], []], [], ["a"]],
                     [r'EEG_manual_100-25cha_0noise_75', 1000, [[], [], [], [], [], []], [], ["a"]],
                     # B
                     [r'EEG_manual_100-25cha_1noise_100', 10, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_1000', 10, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_100', 1000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_1000', 1000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_100', 2000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_1000', 2000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_100', 3000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_1000', 3000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     # C
                     [r'EEG_manual_100-25cha_1noise_10', 10, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_10', 1000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_30', 10, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_30', 1000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_50', 10, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_50', 1000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_75', 10, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     [r'EEG_manual_100-25cha_1noise_75', 1000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                     # D
                     [r'Combinations\EEG_manual_100-25cha_noise_10noise_100', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_30noise_100', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_50noise_100', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_75noise_100', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_10noise_1000', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_30noise_1000', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_50noise_1000', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     [r'Combinations\EEG_manual_100-25cha_noise_75noise_1000', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                     # E
                     [r'EEG_manual_100-25cha_2noise_10', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_10', 1000, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_30', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_30', 1000, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_50', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_50', 1000, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_75', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                     [r'EEG_manual_100-25cha_2noise_75', 1000, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]]]

sim_test_data = [[r'EEG_manual_100-25cha_0noise_100', 10, [[], [], [], [], [], []], [], ["a"]],
                 [r'EEG_manual_100-25cha_0noise_1000', 10, [[], [], [], [], [], []], [], ["a"]],
                 [r'EEG_manual_100-25cha_1noise_100', 4000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["b"]],
                 [r'EEG_manual_100-25cha_1noise_1000', 4000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["b"]],
                 [r'EEG_manual_100-25cha_1noise_30', 2000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                 [r'EEG_manual_100-25cha_1noise_75', 2000, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA'], ["c"]],
                 [r'Combinations\EEG_manual_100-25cha_noise_30noise_1000', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                 [r'Combinations\EEG_manual_100-25cha_noise_75noise_100', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["d"]],
                 [r'EEG_manual_100-25cha_2noise_100', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]],
                 [r'EEG_manual_100-25cha_2noise_1000', 10, [['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB'], ['ELA','ELB']], ['ELA','ELB'], ["e"]]]

In [3]:
print(len(sim_training_data))
print(len(sim_test_data))

40
10


## Helper functions

In [4]:
def check_file_exists(data_set, simulation):
    if simulation:
        local_folder = r"D:\Bachelor-Project-Data\Manual_data_28_05_2023"
    else:
        local_folder = r"D:\Bachelor-Project-Data\Kaare\study_1A_mat_simple"
    
    for data in data_set:
        folder_path = local_folder + "\ "[:-1] + data[0]
        
        if not os.path.isfile(folder_path + ".mat"):
            print(folder_path)

In [5]:
check_file_exists(sim_training_data, True)
check_file_exists(sim_test_data, True)

In [6]:
def count_types(data_set):
    catogory_dictonary = {}
    
    # Runs through each data point in the data set
    for data_point in range(len(data_set)):
        
        #Runs through each category type:
        for category_type in data_set[data_point][4]:
            if category_type in catogory_dictonary.keys():
                # Add 1 to the type
                catogory_dictonary.update({category_type: catogory_dictonary.get(category_type) + 1})
            else:
                # Add a new type to the dictonary
                catogory_dictonary.update({category_type: 1})
    return catogory_dictonary

In [7]:
count_types(sim_training_data)

{'a': 8, 'c': 16, 'd': 8, 'e': 8}

In [8]:
count_types(sim_test_data)

{'a': 2, 'b': 2, 'c': 2, 'd': 2, 'e': 2}

# Optimisation

Parameters to optimise:
* Relative procent  {1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00}
* Measure of cross {M_idx, M_max}
* Matrix {MM, UM}
* Number of 10 s. intervals in a row {1, 2, 3, 4, 5, 6}

$7\cdot2\cdot2\cdot6 = 168$ grid search

In [9]:
relative_procents = [1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00]
measures = ['M_idx','M_max']
matrices = ['mm','um']
artifact_intervals = [1, 2, 3, 4, 5, 6]
parameters_op = [measures, matrices, relative_procents, artifact_intervals]

## Optimisation Broken Electrode classifier

In [10]:
def optimise_set(data_set, simulation, parameters):
    # Empty list to the output.
    outputs = [] 
    
    start_cont = timeit.default_timer() # starts timer
    for data in data_set:
        #Running through every data point
        start = timeit.default_timer() # starts timer
        
        # Extracting the information of the data.
        if simulation:
            local_folder = r"D:\Bachelor-Project-Data\Manual_data_28_05_2023"
        else:
            local_folder = r"D:\Bachelor-Project-Data\Kaare\study_1A_mat_simple"
        folder_path = local_folder + "\ "[:-1] + data[0] #S_17/night_1\EEG_raw
        time_start = data[1]
        artifact_labels = data[2]
        broken_labels = data[3]
        
        # Loading the full data.
        raw_all = load_mne.file_to_raw(folder_path + ".mat")
        
        # Cropping it to the 60 seconds + buffer interval.
        raw_all = raw_all.crop(tmin=time_start - 4, tmax = time_start + 64)
        
        if simulation:
            #No preprocess
            raw_all_processed = raw_all.crop(tmin=4, tmax=64)

        else:
            # Preprocess
            raw_all_processed, NaN = load_mne.preprocess(raw_all)
            # Remove bufer interval.
            raw_all_processed = raw_all_processed.crop(tmin=4, tmax=64)
        
        # Prepares to loop through intervals.
        intv_count = 6
        intv_size = 10
        intv_end = intv_count * intv_size + 1 #Etc. 6*10 + 1
        time_intv = [x for x in range(0, intv_end, intv_size)]
        
        # ICA on intervals
        for intv_idx in range(intv_count):
            # Running through each interval
            folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
            
            # Cropping to the interval to perform multiple functions [0, 10], [10, 20] etc.
            raw_crop_int = raw_all_processed.copy().crop(tmin = time_intv[intv_idx], tmax = time_intv[intv_idx + 1])
            
            # Runs multiple ICA for 2 - 15 number of components, both measures and matrices are used.
            ICA_subfunctions.multiple_ica(raw_crop_int, 2, 15, folder_int, matrix=parameters[1], cross_measure=parameters[0])
            
            # Runs multi_plot to get a summary of all 2 - 15 components.
            if type(parameters[1]) is not list and type(parameters[0]) is not list:
                ICA_subfunctions.multi_plot(2, 15, folder_int, matrix_list=[parameters[1]], cross_measure=[parameters[0]])
            else:
                ICA_subfunctions.multi_plot(2, 15, folder_int, matrix_list=parameters[1], cross_measure=parameters[0])
        
        if type(parameters[3]) is not list and type(parameters[2]) is not list and type(parameters[1]) is not list and type(parameters[0]) is not list:
            # Running the artifact classifier with relative procent on each ICA interval
            artifact_intervals = []
            for intv_idx in range(intv_count):
                folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
                artifact_intervals.append(ICA_subfunctions.artifact_classifier(folder_int, parameters[2], cross=parameters[0], matrix=parameters[1]))

            # Running the broken electrode classifier with number of 10 s. intervals in row.
            broken_pred = ICA_subfunctions.broken_channel_classifier(artifact_intervals, broken_interval_size = parameters[3])

            # Output the predictions for the combination of parameters.
            outputs.append([data[0], time_start, parameters[0], parameters[1], parameters[2], parameters[3], artifact_intervals, broken_pred])
        else:
            # Artifact classifier
            for art_int in parameters[3]:
                for rel_pro in parameters[2]:
                    for matrix in parameters[1]:
                        for cross in parameters[0]:
                            # Running the artifact classifier with relative procent on each ICA interval
                            artifact_intervals = []
                            for intv_idx in range(intv_count):
                                folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
                                artifact_intervals.append(ICA_subfunctions.artifact_classifier(folder_int, rel_pro, cross=cross, matrix=matrix))

                            # Running the broken electrode classifier with number of 10 s. intervals in row.
                            broken_pred = ICA_subfunctions.broken_channel_classifier(artifact_intervals, broken_interval_size = art_int)

                            # Output the predictions for the combination of parameters.
                            outputs.append([data[0], time_start, cross, matrix, rel_pro, art_int, artifact_intervals, broken_pred])
        
        # ends timer
        stop = timeit.default_timer() 
        total_time = (stop - start_cont)/60
        time_estimated = (stop - start)*len(data_set)/60
        print(str(total_time)[:4] + "min. Estimated time left with this speed: " + str(time_estimated - total_time)[:4] + "min.")
    return outputs

In [11]:
#training_outputs = optimise_set(training_data, False, parameters_op)
#print(training_outputs)

## Predictions

Simulated training (with preprocessing)

In [12]:
# sim_training_outputs = [['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELE', 'EOGr', 'F3', 'EMGc', 'ERT ', 'ELT', 'ELI', 'ERE', 'EMGl']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EOGr', 'F3', 'EMGc', 'F4', 'ELI', 'ERE', 'EMGl']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'F3', 'EMGc', 'EMGl', 'ELI', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'ELT', 'EMGr', 'F3', 'EMGc', 'ERT ', 'EMGl', 'F4', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 1, [[], ['M2'], [], [], [], ['ELC']], ['M2', 'ELC']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EOGr', 'F3', 'EMGc', 'ERT ', 'ERE', 'EMGl']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'F3', 'EMGc', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'F3', 'EMGc', 'EMGl', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'F3', 'EMGc', 'EMGl', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 2, [[], ['M2'], [], [], [], ['ELC']], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EOGr', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 3, [[], ['M2'], [], [], [], ['ELC']], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'C3', 'M2', 'O2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERT ', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 4, [[], ['M2'], [], [], [], ['ELC']], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M2', 'C4', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 5, [[], ['M2'], [], [], [], ['ELC']], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'C3', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'EOGr', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'M2', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 6, [[], ['M2'], [], [], [], ['ELC']], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'O2'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'ERB', 'ELE', 'ELT', 'ERE', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'ERT ', 'EOGr', 'EMGr', 'ERB', 'ELT', 'ERE', 'EMGl', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'EOGr', 'EMGl', 'EMGr', 'ELE', 'ERB', 'F4', 'F3', 'C4', 'ELT', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'ELE', 'F3', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'O2'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ERT ', 'EOGr', 'EMGr']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ERT ', 'EOGr', 'EMGr', 'ERB']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'EOGr', 'EMGr', 'ERB', 'F3', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ERC', 'EOGr', 'EMGl', 'F3']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'O2'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M1', 'M2', 'O2', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M1', 'M2', 'O2', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'O2', 'EMGc', 'F3', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc', 'ERC', 'F3']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'O2'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M1', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M1', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'M1', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'O2'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'C3', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'O2'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELC']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C4', 'O2', 'EMGc']], ['ELC']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc', 'ERA', 'EOGl', 'F4', 'O2', 'EMGl', 'F3', 'ERC', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ERA', 'EOGl', 'EMGl', 'C3', 'F4', 'F3', 'M2', 'ERC', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ERA', 'EOGl', 'EMGl', 'O2', 'ERB', 'F3', 'ERC', 'ELA', 'ELB', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 1, [['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'EOGl', 'O2', 'F3', 'ELT', 'ERA', 'ELA', 'ELB', 'EMGl', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'EMGc', 'ERA', 'EOGl', 'F4', 'O2', 'EMGl', 'F3', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C4', 'O2', 'EMGc', 'ERA', 'EOGl', 'EMGl', 'C3', 'F4', 'F3', 'M2', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ERA', 'EMGl', 'O2', 'ERC', 'ELB']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 2, [['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'O2', 'F3', 'ERA', 'ELB']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'ERA', 'F4', 'O2', 'EMGl', 'F3', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ERT ', 'EOGr', 'M1', 'C4', 'O2', 'ERA', 'EMGl', 'F4', 'F3', 'M2']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGl', 'O2', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 3, [['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'M1', 'C4', 'ERA', 'EMGl']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERT ', 'M1', 'O2', 'ERA', 'EMGl', 'M2']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 4, [['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGr', 'M1', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERT ', 'M1', 'EMGl']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGr', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 5, [['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERT ']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGr', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 6, [['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELA', 'ELB', 'ERT ', 'ERE', 'EMGr', 'O1', 'ERB', 'ERC', 'EMGc']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ERB', 'ERT ', 'ERE', 'EMGr', 'O1', 'ELA', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELB', 'ERB', 'EMGl', 'EMGr', 'O1', 'ELA', 'EMGc', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELB', 'ERB', 'EMGr', 'O1', 'ELA', 'EMGc', 'ELT']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 1, [[], ['ELC'], [], [], [], []], ['ELC']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELB', 'ERT ', 'ERE', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ERT ', 'ERE', 'O1', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGl', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELB', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 2, [[], ['ELC'], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'ERT ', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELC', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'F4', 'C4', 'O2', 'ERT ', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 3, [[], ['ELC'], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'F3', 'F4', 'C4', 'O2', 'ERT ', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELC', 'F3', 'C4', 'O2', 'ERT ']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'F3', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 4, [[], ['ELC'], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'F4', 'ERT ', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELC', 'O2', 'ERT ']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERT ', 'M2', 'F4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 5, [[], ['ELC'], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['F4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERT ', 'M2', 'F4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERT ', 'ERE', 'M1', 'F4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 6, [[], ['ELC'], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'ERB', 'ELE', 'ERI', 'EOGr', 'EMGl', 'ERT ', 'ERE', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'ERB', 'ERT ', 'EMGl', 'M2', 'ELE', 'EOGr', 'ERE', 'ELC', 'C3']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'ELE', 'EOGr', 'ELC', 'ERT ']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'ERT ', 'ELC', 'ELE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], ['ERE'], []], ['ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ELE', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc', 'ELI', 'M2', 'ELE', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ERE', 'EOGl', 'EMGl', 'ELE', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELA', 'ERE', 'EOGr', 'EOGl', 'ERT ', 'ELE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], ['ERE'], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'EMGc', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'EMGc', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELT', 'ERC', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ELA', 'ERE', 'EOGl', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELB', 'ELT', 'ERC', 'M1', 'O1', 'C4', 'EMGc', 'ERE', 'EOGr', 'EOGl']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], ['ERE'], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGl', 'M1', 'EMGc']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELT', 'M1', 'O1', 'C4']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELT', 'M1', 'O1', 'C4', 'EOGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], ['ERE'], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['M1', 'EMGc']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['M1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['M1', 'O1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELT', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], ['ERE'], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2', 'EMGc']], ['EMGc']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc']], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELT', 'ELI', 'ERC', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELT', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'O2']], ['ELT', 'C4']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], ['ERE'], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'EMGr', 'M2', 'F3', 'ERA', 'C3', 'ELE', 'F4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 1, [['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'EMGr', 'M2', 'ELA', 'F3', 'C3', 'ELE', 'F4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 1, [['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'M2', 'ELA', 'ERB', 'ERI', 'F3', 'ELE', 'F4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 1, [['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'EMGr', 'M2', 'ERA', 'ERI', 'F3', 'ELA', 'F4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'EMGr', 'M2', 'F3', 'C3']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 2, [['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'EMGr', 'M2', 'ELA', 'F3']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 2, [['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'M2', 'ELA', 'ERB']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 2, [['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGr', 'M2']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'M1', 'C4', 'O2', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 3, [['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'ELI', 'ERE', 'EOGl', 'M1', 'C4', 'O2', 'ELC', 'ELA']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 3, [['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc', 'ELC', 'ERB']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 3, [['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'ERB', 'EOGl', 'M1', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 4, [['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'EOGl', 'M1', 'C4', 'ELC', 'ELA']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 4, [['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc', 'ELC', 'ERB']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 4, [['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ERB', 'ERE', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'EOGl', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 5, [['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'EOGl', 'C4', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 5, [['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELT', 'ERE', 'EOGl', 'M1', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 5, [['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ERE', 'M1', 'C4', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 6, [['ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4']], ['ELT', 'C4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 6, [['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELT', 'ERE', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 6, [['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ERE', 'M1', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ERE', 'O2', 'ERT ', 'F3', 'O1', 'EMGc', 'ELC', 'EMGl', 'M1']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 1, [['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ELI', 'ERA', 'ERT ', 'ERE', 'O2', 'ELC', 'EMGc', 'EOGr', 'EMGl', 'F3', 'M1']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'ELA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC', 'C3', 'EMGl', 'O1', 'M1']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'M2', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'ELA', 'ERT ', 'ERE', 'M1', 'O2', 'EMGc', 'ELC', 'C3', 'ELE', 'EMGl']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], ['F4', 'C4'], [], ['EMGr']], ['F4', 'C4', 'EMGr']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'ERE', 'O2', 'ERT ', 'F3']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 2, [['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'ELI', 'ERT ', 'ERE', 'O2', 'F3']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'ELA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'M2', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'ELA', 'ERT ', 'ERE', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], ['F4', 'C4'], [], ['EMGr']], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 3, [['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'EOGl', 'EMGr', 'F4', 'C4', 'ELI', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F4', 'C4', 'ERT ', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'M2', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F4', 'C4', 'ERT ', 'ERE', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], ['F4', 'C4'], [], ['EMGr']], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 4, [['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'EOGl', 'EMGr', 'F4', 'C4', 'ELI', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'M2', 'C4', 'O2']], ['ELB', 'EOGl', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], ['F4', 'C4'], [], ['EMGr']], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 5, [['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'EMGr', 'F4', 'C4', 'ELI', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'M2', 'C4', 'O2']], ['ELB', 'EOGl', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], ['F4', 'C4'], [], ['EMGr']], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'EMGr', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 6, [['ELB', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'EMGr', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'M2', 'C4', 'O2']], ['ELB', 'EOGl', 'EMGr', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], ['F4', 'C4'], [], ['EMGr']], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'O2', 'EMGc', 'ERE', 'F3', 'ELA', 'ERI']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 1, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'ELB', 'ELE', 'ERA', 'ERT ', 'O2', 'EMGc', 'ERE', 'ERI', 'F3', 'ELA']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELE', 'ERA', 'ERT ', 'C4', 'ELA', 'ERE', 'ERI']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 1, [['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'C4', 'ELA', 'ERT ']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 1, [[], [], ['O2'], ['C4'], [], []], ['O2', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'EMGr', 'F4', 'C4', 'ELB', 'ERA', 'ERT ', 'EOGl', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 2, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'EOGl', 'EMGr', 'F4', 'C4', 'ELB', 'ERA', 'ERT ', 'O2', 'ERI']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ERA', 'ERT ', 'C4', 'ELA', 'ERI']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 2, [['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELI', 'ERA', 'ERE', 'C4', 'ELA']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 2, [[], [], ['O2'], ['C4'], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELI', 'EMGr', 'F4', 'C4', 'ELB', 'EOGl', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 3, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'EOGl', 'EMGr', 'F4', 'C4', 'ELB', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'EOGl', 'EMGr', 'F4', 'O2', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 3, [['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'EOGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 3, [[], [], ['O2'], ['C4'], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['EMGr', 'C4', 'ELB', 'EOGl', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 4, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'EOGl', 'EMGr', 'F4', 'C4', 'ELB', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'O2', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 4, [['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F4', 'O2', 'EMGc', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 4, [[], [], ['O2'], ['C4'], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['EMGr', 'C4', 'ELB', 'EOGl', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 5, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'F4', 'C4', 'ELB', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'O2', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 5, [['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F4', 'O2', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 5, [[], [], ['O2'], ['C4'], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['EMGr', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 6, [['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERI', 'EOGl', 'EMGr', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 6, [['ELC', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 6, [[], [], ['O2'], ['C4'], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ELB', 'EOGr', 'EMGl', 'C3', 'F4', 'ELE', 'ERE', 'F3', 'M2', 'O2', 'ERC', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'EMGc']], ['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ELB', 'C3', 'F4', 'F3', 'EOGr', 'C4', 'ELI', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc', 'ELB', 'EOGr', 'EMGl', 'C3', 'F4', 'ELE', 'F3', 'C4', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ELB', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'ELE', 'ELI', 'ERB', 'O2', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'M1', 'O1', 'EMGc', 'ELB', 'EOGr', 'C3', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'EMGc']], ['ELA', 'ELT', 'ERA', 'EOGl', 'M1', 'O1', 'EMGc', 'ELB', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ELB', 'EOGr', 'C3', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ELB', 'F3', 'C3', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'EOGl', 'M1', 'O1', 'EMGc', 'EOGr', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'EMGc']], ['ELA', 'ELT', 'ERA', 'EOGl', 'M1', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'M1', 'O1', 'EMGc', 'EOGr', 'C3', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'EOGl', 'M1', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'O1', 'EMGc', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'EMGc']], ['ELA', 'EOGl', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'O1']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'ELC', 'ERB', 'EMGr', 'ERC', 'ERE', 'EMGl', 'C4', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'EOGl'], ['ELA']], ['ELA', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ERB', 'ERE', 'EMGr', 'EMGl', 'F4', 'ELE', 'ERA', 'M1']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ERE', 'EMGr', 'EMGl', 'F4', 'ERA', 'M1']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'ELC', 'ERB', 'ERC', 'ERA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'EOGl'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ERB', 'ERE']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'ERE', 'EMGl', 'ERA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'F3', 'O1', 'M2', 'ELC', 'ERB', 'ERC']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'EOGl'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGl', 'O1', 'M2', 'C4', 'O2', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'M2', 'ERC']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'EOGl'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELC', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2']], ['ELC', 'ELT', 'ERB', 'ERC', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'EOGl'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['O1', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ERC', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'EOGl'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['O1']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'O1', 'M2'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2']], ['ELT', 'ERC', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1']], ['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc', 'ELB', 'ELT', 'ELI', 'EOGl', 'F4', 'C4', 'EMGl', 'C3', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3']], ['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'ELI', 'F4', 'C4', 'M2', 'ERB', 'EOGl']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2']], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'ERA', 'ERB', 'C4', 'EMGl', 'C3', 'ERC', 'ERI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'C4', 'ELE', 'C3', 'ERI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], [], [], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 1, [['ELA'], [], ['ELA'], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1']], ['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc', 'ELB', 'ELI', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3']], ['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2']], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'ERA', 'ERB', 'C4', 'C3']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'C4', 'C3']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], [], [], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 2, [['ELA'], [], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1']], ['ELA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc', 'ELB', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3']], ['ELA', 'EMGr', 'M1', 'O1', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2']], ['ELA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'C4', 'C3']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 3, [['ELA'], [], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1']], ['ELA', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2']], ['ELA', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'C4']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 4, [['ELA'], [], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1']], ['ELA', 'M1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2']], ['ELA', 'EMGr', 'M1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'EMGr', 'M1', 'EMGc', 'ELB', 'C4']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 5, [['ELA'], [], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ERA', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2']], ['ELA', 'EMGr', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 6, [['ELA'], [], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EMGl', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELB', 'ERC', 'EOGl', 'F3', 'ELC', 'EMGl', 'ERE', 'ERI', 'M1']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA', 'ELI', 'C4'], ['ELA', 'EMGr'], ['ELA'], ['ELA']], ['ELA', 'ELI', 'C4', 'EMGr']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ELA', 'ELB', 'O2', 'ERC', 'ERI', 'EMGr', 'EMGl', 'C3', 'ELE']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 1, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ELB', 'ERC', 'EMGl', 'O2', 'ERI', 'EMGr', 'F4', 'C3', 'ELE']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EMGl', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELB', 'EOGl', 'F3', 'ELC']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA', 'ELI', 'C4'], ['ELA', 'EMGr'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ELB', 'EMGr']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 2, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ELB', 'ERC', 'ERI', 'EMGr']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EMGl', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'EOGr', 'O1', 'C4', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA', 'ELI', 'C4'], ['ELA', 'EMGr'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'C4', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 3, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EMGl', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'O2']], ['ELA', 'ELT']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA', 'ELI', 'C4'], ['ELA', 'EMGr'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELI', 'ERB', 'EOGl', 'O1', 'M2', 'C4', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 4, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELI', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EMGl', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA', 'ELI', 'C4'], ['ELA', 'EMGr'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELI', 'ERB', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 5, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELI', 'ERB', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EMGl', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA', 'ELI', 'C4'], ['ELA', 'EMGr'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELI', 'ERB', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 6, [['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELI', 'ERB', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'ELI', 'EOGr', 'EMGr', 'C4', 'EMGc', 'ERB', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'O1', 'F4']], ['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc', 'ERB', 'ELI', 'C4', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELI', 'ERI', 'EOGr', 'C4', 'ERB', 'ERE', 'EMGl', 'ERC', 'O2']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc', 'ELI', 'ERI', 'EOGr', 'EMGr', 'C4', 'ERB', 'M1', 'M2', 'EMGl', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], [], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 1, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 1, [[], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'ELI', 'EOGr', 'EMGr', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'O1', 'F4']], ['ELA', 'ELB', 'M2', 'F4', 'EOGr', 'EMGr', 'M1', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELI', 'EOGr', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4']], ['ELA', 'ELB', 'EOGl', 'C3', 'O1', 'F4', 'EMGc', 'ELI', 'ERI', 'EOGr', 'EMGr', 'C4', 'M1', 'M2']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 2, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 2, [[], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGr', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'O1', 'F4']], ['ELA', 'M2', 'M1', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc', 'ELI', 'EOGr', 'C4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4']], ['ELA', 'ELB', 'EOGl', 'C3', 'O1', 'EMGc', 'EOGr', 'EMGr', 'C4', 'M1']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 3, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 3, [[], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'F4', 'EMGr', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'O1', 'F4']], ['ELA', 'M1', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'M1', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4']], ['ELA', 'ELB', 'EOGl', 'C3', 'O1', 'EMGr', 'C4', 'M1']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 4, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 4, [[], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'O1', 'F4']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'M1', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4']], ['ELA', 'ELB', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 5, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 5, [[], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'F4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGl', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'O1', 'F4']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'M1', 'M2', 'F4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], [], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 6, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 6, [[], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'ELE', 'ERB', 'EOGl', 'EMGr', 'M2', 'EMGc', 'ELB', 'ERA', 'O2', 'F4', 'C4', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB'], ['ELA'], ['ELA', 'F3'], ['ELA'], ['ELA'], ['ELA']], ['ELA', 'ELB', 'F3']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc', 'ELA', 'ERB', 'EMGr', 'M1', 'F3', 'ERI', 'EOGr', 'F4', 'ELE', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc', 'ERB', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'ERI', 'EMGl', 'ELE']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 1, [[], [], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'F3', 'O1', 'ERB', 'EOGl', 'M2', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB'], ['ELA'], ['ELA', 'F3'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc', 'ELA', 'ERB', 'F3', 'ERI']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc', 'ERB', 'F3', 'C4', 'ERI']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 2, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA', 'ELC', 'ELT', 'ELI', 'O1', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB'], ['ELA'], ['ELA', 'F3'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'ELI', 'ERC', 'EOGl', 'O1', 'M2', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'O1', 'M2', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 3, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA', 'ELC', 'ELT', 'ELI']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB'], ['ELA'], ['ELA', 'F3'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'EOGl', 'O1', 'M2', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'ELI', 'O1', 'M2', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 4, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA', 'ELI']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB'], ['ELA'], ['ELA', 'F3'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'EOGl', 'O1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'ELI', 'O1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 5, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB'], ['ELA'], ['ELA', 'F3'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'EOGl', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELC', 'ELT', 'ELI', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 6, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'ELI', 'ERA', 'ERB', 'ERE', 'M2', 'C4', 'EMGc', 'EMGr', 'O2', 'ERT ']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 1, [['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ERA', 'ERB', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERE', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'ERA', 'ERB', 'O1', 'C4', 'EMGc', 'O2', 'ELB', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M2']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ELT', 'ERB', 'ERE', 'O1', 'C4', 'C3', 'O2', 'ELI', 'EMGl']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 1, [['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc', 'ELT', 'F3', 'C3', 'O1', 'C4', 'ELB', 'ELE', 'EMGr', 'O2', 'ELI', 'ELC']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 1, [[], [], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'M1', 'F3', 'C3', 'O1', 'F4', 'ELI', 'ERA', 'ERE', 'C4', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 2, [['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ERA', 'ERB', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERE', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'M1', 'C3', 'F4', 'ERA', 'O1', 'C4', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'EMGc', 'O1', 'C4', 'C3', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 2, [['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'EMGc']], ['ELA', 'ERE', 'EOGl', 'M1', 'F4', 'EMGc', 'ELT', 'F3', 'O1', 'C4', 'ELB', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 2, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'M1', 'F3', 'O1', 'F4', 'ELI', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 3, [['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ERA', 'ERB', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERE', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'M1', 'O1']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EMGr', 'M1', 'F3', 'F4', 'EMGc', 'O1']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 3, [['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'EMGc']], ['ELA', 'EOGl', 'M1', 'EMGc', 'ELT', 'F3', 'O1', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 3, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'F3', 'O1', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 4, [['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ERA', 'ERB', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERE', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'EMGr', 'M1', 'F3', 'F4', 'EMGc', 'O1']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 4, [['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'EMGc']], ['ELA', 'EMGc', 'ELT', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 4, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'F3', 'O1', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 5, [['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ERA', 'ERB', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERE', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'F3', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 5, [['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'EMGc']], ['ELA', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 5, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'F3', 'F4']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 6, [['ELA', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ERA', 'ERB', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERE', 'M1', 'F4', 'C4', 'EMGc'], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 6, [['ELA', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'EMGc']], ['ELA', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 6, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M2', 'EMGc', 'C4', 'O2', 'ELT']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 1, [['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'C4', 'EMGc', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 1, [['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'ELB', 'ELC', 'ERI', 'EOGr', 'O1', 'C4', 'EMGc', 'O2', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'EMGc']], ['ELA', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'F4', 'ELC', 'ERA', 'EOGl', 'M2', 'C4', 'ELT']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 2, [['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'F3', 'M2', 'ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 2, [['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'C3', 'M2', 'ELB', 'ELC', 'EOGr', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'F3', 'O1', 'F4', 'ELC', 'ERA', 'C4', 'ELT']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 3, [['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ERE', 'EOGl', 'F3', 'M2', 'ELA', 'ERA', 'ERC']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 3, [['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'F3', 'M2', 'EOGr', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'ERA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 4, [['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ERE', 'EOGl', 'F3']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 4, [['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELT', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'M2', 'EOGr', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'EMGc']], ['ELA', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 5, [['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['EOGl', 'F3']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 5, [['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ERA', 'EOGl', 'F3', 'M2', 'EOGr', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'EMGc']], ['ELA', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 6, [['ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['EOGl', 'F3']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 6, [['ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ERA', 'F3', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 1, [['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'ELT', 'ERI', 'EOGl', 'F4', 'ELE', 'ELB', 'ERA', 'ELA', 'F3', 'C3']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 1, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'ERI', 'EOGl', 'F4', 'EMGc', 'F3', 'ELB', 'ERA', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'EOGl', 'F4', 'EMGc', 'C3', 'ELB', 'ERA', 'EOGr', 'F3']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 1, [['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'EOGl', 'EMGl', 'F4', 'EMGc', 'ELC', 'C3', 'ELB', 'ERA', 'EOGr', 'F3', 'ELA', 'ERC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 2, [['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'ELT', 'ERI', 'F4', 'ELB', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 2, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'ERI', 'EMGc', 'ELB', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'F4', 'EMGc', 'ELB', 'F3']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 2, [['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'EOGl', 'EMGl', 'F4', 'EMGc', 'ELC', 'ELB', 'ERC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 3, [['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 3, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'M2', 'C4', 'O2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'EMGl', 'M2', 'EMGc', 'F3']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 3, [['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2', 'O2', 'EOGl', 'EMGc']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 4, [['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'M1', 'M2', 'O2', 'ELC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 4, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2', 'C4', 'O2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 4, [['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2', 'O2', 'EOGl']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 5, [['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 5, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 5, [['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'O2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 6, [['ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 6, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGr', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 6, [['ERB', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'O2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'F4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc', 'ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'F4', 'O2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc', 'ELA', 'ERI', 'EMGr', 'F3', 'C4', 'EOGl', 'O1', 'ERA', 'F4', 'EOGr', 'O2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'ELC', 'ELT', 'ERI', 'EMGr', 'F3', 'C4', 'EOGl', 'EMGc', 'ERA', 'EOGr', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'O2'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O2']], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'ELA', 'ELT', 'ERT ', 'ERI', 'EMGl', 'F3', 'EOGl', 'C4', 'EMGc', 'EOGr', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], ['ERE']], ['ERE']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'F4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'M2', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'EMGr', 'F3', 'C4', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'ELT', 'EMGr', 'F3', 'C4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'O2'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O2']], ['ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'ELA', 'ELT', 'ERI', 'EMGl', 'F3', 'EOGr']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'F4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELT', 'ERB', 'ERC', 'ERE', 'M1', 'M2', 'O1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELT', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'O1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'O2'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O2']], ['ERB', 'ERC', 'ERE', 'M1', 'O2', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'F4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELT', 'ERB', 'ERC', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERB', 'ERC', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERB', 'ERC', 'ERE', 'M1', 'O1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'O2'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O2']], ['ERB', 'ERC', 'ERE', 'M1', 'O2', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'F4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELT', 'ERB', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERB', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERE', 'M1', 'O1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'O2'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O2']], ['ERE', 'M1', 'O2', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'F4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELT', 'ERB', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4'], ['ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERB', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'O2'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'O2']], ['ERE', 'M1', 'O2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc', 'ELE', 'ERA', 'ERB', 'F4', 'ELC', 'O2', 'ELB', 'M2']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc', 'ELE', 'ERA', 'ERB', 'F4', 'F3', 'ELB', 'ELC']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'ELE', 'ERA', 'ERB', 'ERT ', 'F4', 'C4', 'EMGc', 'F3', 'ELB', 'ELC', 'EMGl', 'O2', 'ERC', 'M2']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ELE', 'ERB', 'ERT ', 'F4', 'ELB', 'ELC', 'EMGl', 'O2']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 1, [[], [], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ELE', 'ERA', 'ERB', 'F4', 'ELC', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ELE', 'ERB', 'F4', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'ELE', 'ERB', 'F4', 'C4', 'EMGc', 'ELB', 'ELC', 'EMGl']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc', 'ELE', 'ERB', 'ELB', 'ELC']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 2, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ERB', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'ERB', 'F4', 'C4', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc', 'ERB', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 3, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ERB', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'O1', 'ERB', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 4, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'O1', 'ERB', 'F4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELI', 'EOGl', 'M1', 'O1', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 5, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['EOGl', 'EMGr', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'O1']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELI', 'EOGl', 'M1', 'O1', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 6, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'M1', 'F3', 'EMGc', 'ERI', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'ERB', 'M1', 'F3', 'C4', 'EMGc', 'ERI', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4', 'ERA', 'ERE', 'F3', 'O1', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'M1', 'F3', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'ERB', 'M1', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4', 'ERA', 'ERE', 'F3', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F4', 'C4', 'ERE', 'F3', 'O1', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELT', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4', 'ERE', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C4', 'ERE', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'ERB', 'EOGl', 'EMGl', 'C4', 'ERE', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'ERB', 'EOGl', 'C4', 'ERE', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'EOGl', 'C4', 'ERE', 'O1']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'EOGl', 'C4', 'ERE', 'O1']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'O1']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'EOGl', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4']], ['ELA', 'ELI', 'EOGl', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'ELC', 'ELI', 'ERA', 'EMGl', 'O1', 'EMGc', 'F4', 'EOGr', 'EMGr', 'C4', 'M2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'O1'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'ELC', 'ELI', 'ERA', 'EMGl', 'O1', 'F4', 'EMGc', 'EOGr', 'EMGr', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'ELC', 'ELI', 'EMGl', 'O1', 'EMGc', 'ERA', 'ERE', 'M2', 'F4']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'ELC', 'ELI', 'EMGl', 'O1', 'EMGc', 'ERA', 'ERC', 'M2', 'F4', 'ELT', 'C4', 'EMGr', 'ERE']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'ELC', 'ERA', 'EMGl', 'O1', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'O1'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELT', 'ERT ', 'M1', 'F3', 'ERA', 'EMGl', 'O1', 'EMGc', 'EOGr', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'F3', 'O2', 'ELC', 'O1', 'EMGc', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'F3', 'O2', 'ELC', 'EMGc', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'F3', 'ELC', 'ERA', 'EMGl', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'O1'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELT', 'ERT ', 'F3', 'ERA', 'EMGl', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'F3', 'O2', 'ELC', 'EMGc', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'F3', 'O2', 'ELC', 'EMGc', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'O1'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELT', 'ERT ', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'O2', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'O2', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'O1'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ERT ', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EMGl', 'F3', 'O1'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ERT ']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELB', 'ELC', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'EOGl']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELB', 'ELC', 'ELT', 'ERC', 'ERT ', 'F3', 'C3', 'EOGl', 'F4']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ELB', 'ELC', 'ERC', 'C3', 'ERE', 'ELI', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc', 'ELB', 'ELC', 'ERC', 'ERT ', 'C3', 'ELI', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 1, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELB', 'ELC', 'F3', 'EOGl']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELB', 'ELC', 'ELT', 'ERT ', 'F3', 'EOGl']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'ELB', 'ELC']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'M2', 'EMGc', 'ELC', 'ERT ', 'ELI']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 2, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'O1', 'ELB', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERI', 'O1', 'ELB', 'ELT', 'F3', 'EOGl']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'ELB']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERI', 'F3', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 3, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'O1', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA', 'ELT', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERI', 'F3', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 4, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'O1', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 5, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELE', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERT ', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'F3', 'O2', 'EMGc']], ['ELA', 'ERA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 6, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2', 'ELC', 'ERB', 'C4', 'EOGr', 'F3', 'O1', 'EMGc', 'M1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERB', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2', 'ELC', 'ERB', 'C4', 'EOGr', 'EMGc', 'M1', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2', 'ELC', 'ERB', 'C4', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2', 'ELC', 'ERB', 'ERT ', 'C4', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELE'], ['ELA', 'ELE', 'ERA'], ['ELA']], ['ELA', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 1, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], [], [], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'C3', 'O2', 'ERB', 'C4', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERB', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'O2', 'ERB', 'C4', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'C3', 'O2', 'ERB', 'C4', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'C3', 'O2', 'ERB', 'ERT ', 'C4', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELE'], ['ELA', 'ELE', 'ERA'], ['ELA']], ['ELA', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 2, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'O2', 'ERB', 'C4', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERB', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'O2', 'ERB', 'C4', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'C3', 'O2', 'ERB', 'C4', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'C3', 'O2', 'ERB', 'ERT ', 'C4', 'F3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELE'], ['ELA', 'ELE', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 3, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERB', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'C4', 'O2']], ['ELA', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'F3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O2']], ['ELA', 'ELT', 'ELE', 'F3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELE'], ['ELA', 'ELE', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 4, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERB', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'C4', 'O2']], ['ELA', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O2']], ['ELA', 'ELT', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELE'], ['ELA', 'ELE', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 5, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C3', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERB', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'C4', 'O2']], ['ELA', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'C3', 'O2']], ['ELA', 'ELT', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELE'], ['ELA', 'ELE', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 6, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4', 'ELE', 'M1', 'C3', 'O2', 'ERE', 'EOGl', 'F3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'ELE', 'C3', 'C4', 'O2', 'M1', 'ERE', 'F3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4', 'ELE', 'ELI', 'EMGr', 'F3', 'C3', 'O2', 'ERE', 'F4', 'ELB']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'ELE', 'ELI', 'EMGr', 'O2', 'ERE', 'ELT', 'ERB', 'ELB', 'F3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 1, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 1, [[], ['ELA'], [], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4', 'ELE', 'M1', 'C3', 'O2', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'ELE', 'C3', 'C4', 'O2', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'C4', 'ELE', 'ELI', 'C3', 'O2', 'ERE', 'F4']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'ELI', 'O2', 'ERE', 'ERB']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 2, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 2, [[], ['ELA'], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'EOGr', 'O1', 'C4', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1']], ['ELA', 'ERA', 'O1', 'ELE', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'O2']], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'C4', 'ELE', 'C3', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'C4', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 3, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 3, [[], ['ELA'], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'EOGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1']], ['ELA', 'ERA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'O2']], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'O1', 'C4', 'C3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'ERC', 'C3', 'O1', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 4, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 4, [[], ['ELA'], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1']], ['ELA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'O2']], ['ELA', 'ERC', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'ERC', 'C3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 5, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 5, [[], ['ELA'], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1']], ['ELA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'O2']], ['ELA', 'ERC', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'C3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 6, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 6, [[], ['ELA'], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc', 'EMGr', 'C3', 'C4', 'ELI', 'O1', 'ERB', 'ERC', 'M1', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGc'], ['ELA', 'ELI', 'ERA', 'EOGl', 'EMGc'], ['ELA', 'ELE', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2', 'EMGc', 'ELB', 'ELC', 'ERA', 'EMGr', 'C3', 'F4', 'ELI', 'ERB', 'ERC', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc', 'ELI', 'EMGr', 'M1', 'C4', 'C3', 'ERC', 'ERE', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc', 'ERC', 'EMGr', 'ELI', 'O2', 'ERE', 'F3', 'C4', 'ERI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 1, [[], [], ['ELA'], [], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 1, [[], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'F4', 'EMGc', 'EMGr', 'C3', 'C4', 'ELI', 'ERC']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGc'], ['ELA', 'ELI', 'ERA', 'EOGl', 'EMGc'], ['ELA', 'ELE', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc', 'ELB', 'ELC', 'ERA', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'F4', 'EMGc', 'ELI', 'EMGr', 'M1', 'C4', 'C3', 'ERC', 'ERE']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'C4']], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'F4', 'EMGc', 'ERC', 'EMGr', 'ELI', 'ERE', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 2, [[], [], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 2, [[], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGc', 'EMGr', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGc'], ['ELA', 'ELI', 'ERA', 'EOGl', 'EMGc'], ['ELA', 'ELE', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'EOGl', 'EMGc', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F4', 'EMGc', 'ELI', 'EMGr', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'C4']], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGl', 'F4', 'EMGc', 'ERE']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 3, [[], [], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 3, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGc', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGc'], ['ELA', 'ELI', 'ERA', 'EOGl', 'EMGc'], ['ELA', 'ELE', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F4', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'C4']], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 4, [[], [], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 4, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGc'], ['ELA', 'ELI', 'ERA', 'EOGl', 'EMGc'], ['ELA', 'ELE', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'C4']], ['ELA', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 5, [[], [], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 5, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'ELE', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELE', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELE', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGc'], ['ELA', 'ELI', 'ERA', 'EOGl', 'EMGc'], ['ELA', 'ELE', 'EOGl', 'F3', 'F4', 'C4']], ['ELA', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELE', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'C4']], ['ELA', 'ELE', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 6, [[], [], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 6, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'ELI', 'ERA', 'ERC', 'ERE', 'EMGc', 'ELT', 'ELE', 'EMGl', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'EOGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1'], ['ELA', 'ELB', 'ELE', 'EOGl', 'F3', 'M2', 'C4']], ['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGc', 'ELT', 'ERC', 'ELE', 'M2']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2', 'ELC', 'ELI', 'ERA', 'M1', 'O1', 'EMGl', 'M2', 'EMGc', 'ELT', 'ELE', 'ERC', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'ELC', 'ELI', 'ERA', 'ERB', 'EMGl', 'C3', 'EMGc', 'ELT', 'ELE', 'ERC', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], [], [], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], ['ELA'], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'ERE', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'EOGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1'], ['ELA', 'ELB', 'ELE', 'EOGl', 'F3', 'M2', 'C4']], ['ELA', 'ELB', 'EOGl', 'F3', 'O1', 'C4', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'ERA', 'O1', 'M2', 'EMGc', 'ERC']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], [], [], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], ['ELA'], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'EOGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1'], ['ELA', 'ELB', 'ELE', 'EOGl', 'F3', 'M2', 'C4']], ['ELA', 'ELB', 'EOGl', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'O1']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], [], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], ['ELA'], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'EOGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1'], ['ELA', 'ELB', 'ELE', 'EOGl', 'F3', 'M2', 'C4']], ['ELA', 'ELB', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], [], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], ['ELA'], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'EOGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1'], ['ELA', 'ELB', 'ELE', 'EOGl', 'F3', 'M2', 'C4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], [], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], ['ELA'], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'EOGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1'], ['ELA', 'ELB', 'ELE', 'EOGl', 'F3', 'M2', 'C4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], [], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], ['ELA'], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EOGr', 'ELC', 'ERA', 'F3', 'M2', 'ERC', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERE', 'C3'], ['ELA', 'ELB', 'ELE', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ELE', 'ERE', 'C3', 'M1', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'ERC', 'F4', 'O2', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'F4', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'ERB', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2', 'F4', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2', 'ERC', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], [], ['ELA'], ['ELA'], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EOGr', 'ELC', 'ERA', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERE', 'C3'], ['ELA', 'ELB', 'ELE', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ELE', 'C3', 'M1', 'ERA', 'EOGr', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'F4', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'EMGl', 'M1', 'C3', 'O2', 'ERA', 'ERB', 'EOGr', 'F3', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], [], ['ELA'], ['ELA'], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EOGr', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERE', 'C3'], ['ELA', 'ELB', 'ELE', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'M1', 'ERA', 'EOGr', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELA', 'ELB', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'F4', 'ERA', 'EOGr', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'C3', 'ERA', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'EMGl', 'EMGr', 'M1', 'O2', 'EOGr', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERE', 'C3'], ['ELA', 'ELB', 'ELE', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'M1', 'ERA', 'EOGr', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELA', 'ELB', 'M1', 'O2', 'ERA', 'EOGr', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'EMGr', 'M1', 'O2', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERE', 'C3'], ['ELA', 'ELB', 'ELE', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELA', 'ELB', 'M1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'M1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERE', 'C3'], ['ELA', 'ELB', 'ELE', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2']], ['ELA', 'ELB', 'M1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr', 'EMGl', 'F4', 'ERB', 'ERC', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'C3']], ['ELA', 'ELB', 'C3', 'C4', 'EMGc', 'ERA', 'EOGr', 'EMGl', 'ERI', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr', 'EMGl', 'F4', 'M2', 'ERB', 'ERC', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr', 'EMGl', 'M1', 'O1', 'ERC', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 1, [[], [], ['ELA', 'ELB'], ['ELA', 'ELB'], [], []], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB', 'ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'C3']], ['ELA', 'ELB', 'C3', 'EMGc', 'ERA', 'ERI']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'C3', 'EMGc', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 2, [[], [], ['ELA', 'ELB'], ['ELA', 'ELB'], [], []], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB', 'ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'C3']], ['ELA', 'ELB', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 3, [[], [], ['ELA', 'ELB'], ['ELA', 'ELB'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'C3']], ['ELA', 'ELB', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 4, [[], [], ['ELA', 'ELB'], ['ELA', 'ELB'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'C3']], ['ELA', 'ELB', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 5, [[], [], ['ELA', 'ELB'], ['ELA', 'ELB'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'C3']], ['ELA', 'ELB', 'C3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 6, [[], [], ['ELA', 'ELB'], ['ELA', 'ELB'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4']], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'ERT ', 'EOGr', 'M1', 'EMGc', 'ELB', 'ERI', 'O2', 'ERA', 'ELT', 'C3', 'M2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA', 'C4'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc', 'ERT ', 'EOGr', 'M1', 'F4', 'C4', 'ELA', 'M2', 'O2', 'ELE', 'C3']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc', 'ERT ', 'EOGr', 'M1', 'F4', 'ERI', 'M2', 'O2', 'ELE', 'C3']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4']], ['ELA', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C4', 'M1', 'ELB', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA', 'C4'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc', 'F4', 'C4', 'M2', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc', 'EOGr', 'M1', 'F4', 'M2', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4']], ['ELA', 'ELI', 'ERE', 'EOGl', 'EMGl', 'C4', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA', 'C4'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'F4', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4']], ['ELA', 'ELI', 'ERE', 'EOGl', 'EMGl', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA', 'C4'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELI', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'F3', 'C4', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4']], ['ELA', 'ELI', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA', 'C4'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EMGl', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4'], ['ELA', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA', 'C4'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ERE', 'EMGl', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ERC', 'ERE', 'EMGl', 'EMGr', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1'], ['ELA', 'ELB', 'ELE', 'ERI', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc', 'ELE', 'F3', 'ELB', 'ERC', 'ERT ', 'ERI', 'ERB', 'O1', 'ELT']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELE'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['ELA', 'ERA', 'ERB', 'M1']], ['ELA', 'ELE', 'ERE', 'ERA', 'ERB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc', 'ELA', 'ELB', 'ELT', 'F3', 'F4', 'O2', 'ERC', 'ERT ', 'EOGr', 'ELI', 'ERB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELT', 'ERB', 'ERT ', 'F3', 'F4', 'ERC', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1'], ['ELA', 'ELB', 'ELE', 'ERI', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'O2', 'ELE', 'F3', 'ELB', 'ERT ', 'ERI', 'O1']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELE'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['ELA', 'ERA', 'ERB', 'M1']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc', 'ELA', 'ELB', 'O2', 'ERT ', 'ELI', 'ERB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERT ', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1'], ['ELA', 'ELB', 'ELE', 'ERI', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'O2', 'ELB', 'ERI', 'O1']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELE'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['ELA', 'ERA', 'ERB', 'M1']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1'], ['ELA', 'ELB', 'ELE', 'ERI', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELE'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['ELA', 'ERA', 'ERB', 'M1']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'O1', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1'], ['ELA', 'ELB', 'ELE', 'ERI', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'O2']], ['ELA', 'EMGl', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELE'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['ELA', 'ERA', 'ERB', 'M1']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ERA', 'EMGl', 'EMGr', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'EMGl', 'EMGr', 'O1', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1'], ['ELA', 'ELB', 'ELE', 'ERI', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'O2']], ['ELA', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELE'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['ELA', 'ERA', 'ERB', 'M1']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O2']], ['ELB', 'ELE', 'ERA', 'EMGl', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4']], ['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'ELC', 'ELI', 'EOGl', 'C3', 'F4', 'EMGc', 'ERA', 'ERC', 'O1', 'O2', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'ERC', 'EOGl', 'EMGc', 'ERA', 'M2', 'ELA', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc', 'ERC', 'EOGl', 'M1', 'F3', 'ERA', 'M2', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4']], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGr', 'M1', 'ELC', 'C3', 'F4', 'EMGc', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'ERC', 'EMGc', 'ERA', 'M2', 'ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'EMGc', 'ERC', 'M1', 'F3', 'ERA', 'M2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4']], ['ELA', 'ELB', 'ERB', 'M1', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'M1', 'M2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGr', 'EMGl', 'O1', 'EMGc', 'M1', 'M2']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4']], ['ELB', 'ELE', 'EOGr', 'O1', 'EMGc', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4']], ['ELB', 'EOGr', 'EMGc', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4']], ['ELB', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1', 'C4', 'ELC', 'ELT', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERC', 'EMGl', 'M1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ERE'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB', 'ERE']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'EOGr', 'EMGl', 'EMGr', 'C4', 'ELC', 'ELT', 'O1', 'M2', 'EMGc', 'ELI', 'ERC', 'M1', 'F4', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'ELC', 'ELT', 'EOGr', 'EMGc', 'ELI', 'O1', 'M1', 'F4', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], ['ELA'], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'ELC', 'F3', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ERE'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'EOGr', 'EMGl', 'C4', 'ELC', 'O1', 'EMGc', 'ELI', 'ERC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'EMGl', 'EMGr', 'M2', 'C4', 'ELC', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERI', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ERE'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA', 'ELE', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ERE'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'ERE', 'ERI', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'ERE', 'ERI', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], ['ELB'], ['ELB'], ['ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ERE'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'ERI']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'ERI', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], ['ELB'], ['ELB'], ['ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'C3', 'O1'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'C3', 'O1', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ERE'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ERA', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], ['ELB'], ['ELB'], ['ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], ['ELA'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C3', 'C4', 'O2', 'ELT', 'ERB', 'ELA', 'ERI', 'EMGr']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C3', 'C4', 'ELT', 'ERB', 'ERC', 'ERI', 'O1', 'ELA', 'EMGr']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C3', 'C4', 'O2', 'ELT', 'ERB', 'O1', 'ELA', 'ERI', 'EMGr']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'EOGl', 'C3', 'C4', 'O2', 'ERB', 'ELA', 'ERI', 'O1']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], ['EMGc']], ['EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ', 'C3', 'C4', 'ELA', 'EMGr']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERT ', 'C3', 'C4', 'ERC', 'ERI', 'ELA']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ', 'C4', 'O1']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'M2', 'F4', 'EMGc', 'ELI', 'EOGl', 'C4', 'O1']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], ['EMGc']], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'M2', 'F4', 'EMGc', 'ERT ', 'ELA']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'M1', 'M2', 'F4', 'EMGc', 'ERT ', 'ERI', 'ELA']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'M2', 'F4', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], ['EMGc']], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'M2', 'F4', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], ['EMGc']], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'M2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'M2', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERT ', 'M2', 'F4', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], ['EMGc']], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELB']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'M2']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERT ', 'M2', 'F4']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], ['EMGc']], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'F4', 'O2', 'ELA', 'ELI', 'ERI', 'ELC', 'ERB']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc', 'EOGl', 'M1', 'F4', 'O2', 'ELI', 'EMGl', 'C4', 'ELA', 'ERC', 'ERI', 'C3', 'ERB', 'O1', 'ELC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'EOGl', 'O1', 'O2', 'ELA', 'ELC', 'ELI', 'ERC', 'ERI', 'ERB']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 1, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'EOGl', 'O2', 'ELA', 'ELC', 'ELI', 'ERC', 'EMGl', 'ERI', 'ERB', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 1, [['ELE'], ['ELB'], [], [], ['ELI'], []], ['ELE', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], ['ELI'], []], ['ELI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'C4', 'EMGc', 'ERC', 'EMGl', 'O1', 'F4', 'O2', 'ELI', 'ERI', 'ELC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M2', 'EMGc', 'M1', 'O2', 'ELI', 'EMGl', 'C4', 'ELA', 'ERC', 'ERI', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'O1', 'O2', 'ELA', 'ELC', 'ELI', 'ERC', 'ERI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 2, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'O2', 'ELA', 'ELC', 'ELI', 'ERC', 'EMGl', 'ERB', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 2, [['ELE'], ['ELB'], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'C4', 'EMGc', 'ERC', 'EMGl', 'F4', 'O2', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'M2', 'EMGc', 'O2', 'ELI', 'C4', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'O2', 'ELA', 'ELI', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 3, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc', 'O2', 'ELA', 'ELI', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 3, [['ELE'], ['ELB'], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M1', 'M2', 'C4', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'M2', 'ELI', 'C4']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERE', 'M2', 'EMGc', 'ELI', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 4, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERT ', 'ERE', 'M2', 'EMGc', 'ELI', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 4, [['ELE'], ['ELB'], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERE', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 5, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERE', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 5, [['ELE'], ['ELB'], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'M2']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'M2']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 6, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERE', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 6, [['ELE'], ['ELB'], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], ['ELI'], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'ERA', 'ERB', 'ERC', 'ERE', 'O1', 'M2', 'EOGr', 'EOGl', 'EMGc', 'O2', 'ELI', 'EMGr', 'F3']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'ERA', 'ERB', 'ERC', 'M2', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'EOGr', 'EMGc']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'O1', 'M2', 'EOGr', 'F4', 'EMGc', 'ELI', 'ERI', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'ERA', 'ERC', 'ERE', 'O1', 'M2', 'EOGr', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'ERA', 'ERC', 'M2', 'EOGl', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'ERA', 'ERC', 'EOGl', 'O1', 'M2', 'F4', 'EMGc']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'ERA', 'ERC', 'EOGl', 'M2', 'F4', 'EMGc', 'ELI', 'ERI']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ERI', 'C3', 'F4', 'ERA', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ERI', 'C3', 'F4', 'ERA', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERI', 'M1', 'C3', 'ERA', 'ERC', 'M2', 'F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'M1', 'C3', 'ERA', 'ERC', 'M2', 'ERI']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'F4', 'ERA', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'F4', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'M1', 'ERA', 'M2', 'F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'M1', 'ERA', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'F4', 'ERA', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'F4', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ERA', 'M2', 'F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ERA', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4']], ['F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'EMGl', 'EMGr', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'O2', 'ELE', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'ELC', 'F3', 'ERT ']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELT', 'ELI', 'O2', 'ELE', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'ELC', 'F3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc', 'ERI', 'ELE', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'ERC', 'ERT ', 'ERB', 'F3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'C3', 'O1']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERI', 'ELE', 'EMGr', 'ERB', 'ERC', 'M1', 'ERT ', 'O1']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'ELB', 'ELE', 'ERB', 'ERC', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'EMGc', 'ELB', 'ELT', 'ELE', 'ERB', 'ERC', 'ELC', 'F3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc', 'ELE', 'EOGr', 'ERC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'C3', 'O1']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc', 'ELE', 'EMGr', 'ERB', 'ERC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M2', 'ELB', 'ELE', 'ERB', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M2', 'ELB', 'ELE', 'ERB', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M2', 'ERC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'C3', 'O1']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M2', 'ERC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELT', 'EOGl', 'EMGl', 'ELB', 'ELE']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'EOGl', 'EMGl', 'ELB', 'ELE']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'EMGl']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'C3', 'O1']], ['ELA', 'ELB', 'ELT', 'EOGl', 'EMGl']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELT', 'EOGl', 'EMGl', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'EOGl', 'EMGl', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'EMGl']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'C3', 'O1']], ['ELA', 'ELB', 'EMGl']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELT']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGr', 'C3', 'O1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'ELE', 'ERI', 'EMGl', 'O2', 'EMGr', 'ERA', 'F4']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4', 'ELE', 'ERI', 'EMGl', 'EOGl', 'ERA', 'O1', 'F4', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'ELE', 'F4', 'EMGr', 'ERA', 'O1', 'ERE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'ELE', 'F4', 'EMGr', 'ERA', 'O1']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI'], ['ELA', 'ELB']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB']], ['ELB', 'ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'M1', 'C3', 'M2', 'C4', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB']], ['ELB', 'ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3', 'M2', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3', 'M2', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'C3', 'M2', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'M2', 'C4', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB']], ['ELB', 'ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'C3', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C4']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C4']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EMGr', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C4']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'ELE', 'ERA', 'ERI', 'EMGr', 'ELT', 'EOGr', 'O1']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4', 'ELE', 'ERA', 'F3', 'C3', 'ERI', 'EOGl', 'EMGr', 'O1', 'EOGr', 'O2']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'ELE', 'ERA', 'O1', 'ERI', 'EMGr', 'F4', 'ELT', 'EOGr']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M2', 'C4']], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4', 'ERA', 'EOGl', 'EMGr', 'F4', 'O2', 'ELT', 'O1', 'EOGr']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], [], ['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'ELE', 'ERA']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4', 'ELE', 'ERA', 'F3', 'C3', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'ELE', 'ERA']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M2', 'C4']], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], [], ['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'M2']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'M2', 'F3', 'C3', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4', 'ERA']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M2', 'C4']], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'M2', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], [], ['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'F3', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'C4']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], [], ['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], [], ['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'M1']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELI']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], [], ['ELB'], ['ELB'], ['ELA', 'ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], ['ELA'], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'ELC', 'ERA', 'ERE', 'EOGr', 'F4', 'C4', 'O2', 'O1', 'EMGc', 'M1', 'EOGl']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'ELC', 'ERA', 'C4', 'ERE', 'O1', 'EMGc', 'M1', 'F3', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'ELC', 'EOGr', 'F4', 'C4', 'ERE', 'O1', 'EMGc', 'M1', 'O2', 'ELT']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERT ', 'ERE', 'ERI', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3', 'ELC', 'ELI', 'EMGl', 'C3', 'F4', 'ERE', 'O1', 'C4', 'EMGc', 'ERB', 'ELT', 'O2']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 1, [[], ['ELA', 'ELB'], ['ELA'], ['ELB'], [], ['ELA']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 1, [[], ['ELA', 'ELB'], ['ELB'], [], [], []], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 1, [[], ['ELB'], ['ELA', 'ELB'], [], [], []], ['ELB', 'ELA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ERI', 'EMGl', 'C3', 'ELC', 'ERA', 'ERE', 'EOGr', 'F4', 'C4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EMGl', 'ELC', 'ERA', 'C4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'ELC', 'F4', 'C4', 'ERE', 'O2']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERT ', 'ERE', 'ERI', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F4', 'ERE', 'EMGc', 'O2']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 2, [[], ['ELA', 'ELB'], ['ELA'], ['ELB'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 2, [[], ['ELA', 'ELB'], ['ELB'], [], [], []], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 2, [[], ['ELB'], ['ELA', 'ELB'], [], [], []], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ERI', 'EMGl', 'ELC', 'ERA', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ELC', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'ELC', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERT ', 'ERE', 'ERI', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'M1', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 3, [[], ['ELA', 'ELB'], ['ELA'], ['ELB'], [], ['ELA']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 3, [[], ['ELA', 'ELB'], ['ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 3, [[], ['ELB'], ['ELA', 'ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ERI', 'ELC', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERT ', 'ERE', 'ERI', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'O2']], ['ELA', 'ELB', 'ERI']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 4, [[], ['ELA', 'ELB'], ['ELA'], ['ELB'], [], ['ELA']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 4, [[], ['ELA', 'ELB'], ['ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 4, [[], ['ELB'], ['ELA', 'ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ELC', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERT ', 'ERE', 'ERI', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'O2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 5, [[], ['ELA', 'ELB'], ['ELA'], ['ELB'], [], ['ELA']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 5, [[], ['ELA', 'ELB'], ['ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 5, [[], ['ELB'], ['ELA', 'ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ERT ']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERT ']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERE', 'ERI', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERT ', 'ERI', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERT ', 'ERE', 'ERI', 'M1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'O2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 6, [[], ['ELA', 'ELB'], ['ELA'], ['ELB'], [], ['ELA']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 6, [[], ['ELA', 'ELB'], ['ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 6, [[], ['ELB'], ['ELA', 'ELB'], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'ERC', 'ERE', 'C3', 'ERI', 'O2', 'O1', 'EMGc', 'ELE', 'ERB', 'M2']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EMGl', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'ELC', 'ERE', 'ERI', 'C3', 'EMGc', 'M1']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'ELC', 'ERA', 'ERC', 'ERE', 'ELT', 'EMGr', 'O2', 'F3', 'O1', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4', 'ELC', 'ERA', 'ERC', 'ERT ', 'ELT', 'EMGr', 'C3', 'C4', 'O2', 'F3', 'O1', 'EMGc']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], ['ELB'], ['ELB'], []], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'ERC', 'ERE', 'ERI']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EMGl', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4', 'ERE']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'ERA', 'ERE', 'F3']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EMGl', 'M1', 'F4', 'ERA', 'ERT ', 'C3', 'F3']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], ['ELB'], ['ELB'], []], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EMGl', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EMGl', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EMGl', 'M1', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'M1', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], ['ELB'], ['ELB'], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'EMGl', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EMGl', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], ['ELB'], ['ELB'], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'EMGl']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EMGl', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], ['ELB'], ['ELB'], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'C3', 'F4'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EMGl', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'C3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERI', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], ['ELB'], ['ELB'], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], [], [], ['ELA', 'ELB'], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []]]
print( )

Simulated training (without preprocessing) 63.1min

In [13]:
sim_training_predictions = [['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'ELE', 'F3', 'EMGc', 'ERT ', 'EMGl', 'ERE', 'C4']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'ELE', 'F3', 'EMGc', 'EMGl', 'F4', 'ELI', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELE', 'EOGr', 'F3', 'EMGc', 'EMGl', 'ELT', 'ELI', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'ELT', 'EOGr', 'EMGr', 'F3', 'EMGc', 'EMGl', 'F4']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 1, [[], ['M2'], ['ELC'], [], [], []], ['M2', 'ELC']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'F3', 'EMGc', 'ERT ', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'F3', 'EMGc', 'ELI', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'F3', 'EMGc', 'EMGl', 'ELI', 'ERE']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'O2', 'F3', 'EMGc', 'EMGl']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 2, [[], ['M2'], ['ELC'], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 3, [[], ['M2'], ['ELC'], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'C3', 'M2', 'O2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 4, [[], ['M2'], ['ELC'], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'EMGr', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 5, [[], ['M2'], ['ELC'], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'C3', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'EMGr', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'M1', 'C3', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 1.5, 6, [[], ['M2'], ['ELC'], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ERT ', 'EOGr', 'EMGr', 'ERB', 'ELE', 'C4', 'ELT', 'ERE', 'EMGl']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ERT ', 'EOGr', 'ERB', 'ELE', 'EOGl', 'C4', 'ELT', 'EMGl']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'EOGr', 'EMGl', 'ELE', 'F4', 'ELT', 'ERE', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'EMGc', 'ELE', 'F4', 'ERE', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ERT ', 'EOGr', 'EMGr', 'ERB']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ERT ', 'ERB', 'EOGl']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'EMGl', 'ELE', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'ERC', 'EMGc', 'C4']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M1', 'C3', 'M2', 'O2', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EMGr', 'M1', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M1', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'M1', 'M2', 'O2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'M1', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'M2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EMGl', 'F3', 'M2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELC']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'M2', 'EMGc']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'M2']], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_10', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ERA', 'EOGl', 'EMGl', 'O2', 'F3', 'ERC', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc', 'ELC', 'EOGl', 'EMGl', 'F4', 'O2', 'F3', 'ERC', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 1, [['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'EOGl', 'EMGl', 'O2', 'F3', 'ELT', 'ERC', 'ELB', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 1, [['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'EOGl', 'EMGl', 'O2', 'F3', 'ELT', 'ERT ', 'ELA', 'ELB', 'ERC', 'EMGr']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ERA', 'EOGl', 'EMGl', 'F3', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'C4', 'EMGc', 'ELC', 'EMGl', 'F4', 'F3', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 2, [['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'EOGl', 'EMGl', 'O2', 'ELB']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 2, [['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'ERT ', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F4', 'C4', 'ERA', 'EMGl', 'F3']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C4', 'EMGl', 'F4', 'F3']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 3, [['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'F4', 'C4', 'EMGl', 'O2']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 3, [['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'ERT ']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERT ', 'M1', 'F4', 'C4', 'ERA']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 4, [['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 4, [['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERA', 'M1', 'M2', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERA', 'ERT ', 'EOGr', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 5, [['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 5, [['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERA', 'M1', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EMGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ERT ', 'EOGr', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.0, 6, [['ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'ERT ', 'ERE', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.0, 6, [['ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELI', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELB', 'EMGr', 'O1', 'ELA', 'EMGc', 'ERC', 'EOGl']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELI', 'ERB', 'ERT ', 'EMGr', 'O1', 'EMGc', 'ELA', 'ERC', 'EOGl']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELB', 'ERB', 'ERE', 'O1', 'EMGc', 'ELT', 'EMGr', 'ELA', 'ERC']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ERC', 'ERE', 'ELB', 'EMGl', 'O1', 'EOGr', 'EMGc', 'ELT', 'EMGr', 'ERB']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELB', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELI', 'ERT ', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELB', 'ERE', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'ERE', 'EMGl', 'O1']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2']], ['ELC', 'ERA', 'ERE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERT ', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2']], ['ELC', 'ERE', 'F3', 'F4', 'C4', 'O2', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ERT ', 'EMGl', 'F3', 'F4', 'C4', 'O2', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERT ', 'M1', 'F3', 'M2', 'F4', 'C4', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2']], ['ELC', 'ERE', 'F4', 'C4', 'O2', 'ERT ']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ERT ', 'F4', 'C4', 'O2', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERT ', 'M1', 'F4', 'ERE']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERT ', 'ERE', 'EMGl', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'O2']], ['ERE', 'F4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERT ', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ERA', 'M1', 'F4']], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_30', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'ERB', 'ERT ', 'F3', 'M2', 'ELE', 'EOGr', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'ERB', 'ERT ', 'M2', 'EOGr', 'ELC', 'ELE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'ERB', 'ERT ', 'EMGr', 'EOGr', 'ELC', 'ERE', 'ELE', 'ERA', 'F3']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERB', 'EOGl', 'M2', 'EOGr', 'ELC', 'ERT ', 'ERE', 'ELE', 'F3']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'M2', 'EOGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'M2', 'EOGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'EMGr', 'ELC', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'M1', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'M2', 'EOGr', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc', 'M2', 'EOGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc', 'M2', 'EOGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2']], ['ELA', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'EMGr', 'ELC', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'M1', 'C4', 'O2', 'EOGl', 'EOGr', 'ERE']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O2']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O2']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2']], ['ELT', 'EOGl', 'M1', 'O1', 'O2', 'EMGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2']], ['ELT', 'ERC', 'M1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'M2', 'C4', 'O2', 'EMGc']], ['EMGr', 'M1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc']], ['EMGr', 'M1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2']], ['ELT', 'M1', 'O2', 'EMGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2']], ['ELT', 'ERC', 'M1']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'M2', 'C4', 'O2', 'EMGc']], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc']], ['EMGr']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2']], ['ELT', 'O2']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2']], ['ELT']], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELA', 'ERA', 'EMGl', 'EMGr', 'M2', 'F3', 'C3', 'ELE']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ERA', 'EMGl', 'EMGr', 'M2', 'ERB', 'F3', 'ELI', 'C3', 'ELE']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'EMGr', 'M2', 'ERA', 'F3', 'C3', 'ELE', 'ERC', 'F4', 'ELA']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 1, [['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGl', 'EMGr', 'M2', 'ERA', 'F3', 'ELE', 'ERC', 'F4', 'ELA']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ELA', 'ERA', 'EMGr', 'M2', 'C3']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'ERA', 'EMGr', 'M2', 'ERB', 'F3', 'ELI']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc', 'EMGr', 'ERA']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 2, [['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'ELC', 'EMGr', 'ERA']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'ELA', 'ERA', 'EMGr', 'M2', 'C3']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELA', 'ELC', 'ELT', 'EOGl', 'M1', 'O2', 'ERA', 'EMGr', 'ELI']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 3, [['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELT', 'ELI', 'ERB', 'EOGl', 'M1', 'O1', 'C4', 'ELC', 'EMGr']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELT', 'ELI', 'ERB', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELA', 'ELC', 'ELT', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELC', 'ELT', 'ERB', 'EOGl', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 4, [['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELT', 'ERB', 'M1', 'C4', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELT', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELC', 'ELT', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELC', 'ELT', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 5, [['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELT', 'M1', 'C4', 'ELC']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELT', 'EOGl', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4']], ['ELC', 'ELT', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELC', 'ELT', 'M1']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.0, 6, [['ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ERA', 'ERB', 'ERT ', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'C4']], ['ELT', 'M1', 'C4']], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_50', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ERA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC', 'EOGr']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'ELA', 'ERA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ERA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC', 'EOGr', 'M1']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ERA', 'ERT ', 'ERE', 'M1', 'O2', 'O1', 'EMGc', 'ELE', 'EMGl']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 1, [[], [], ['ELI'], [], [], []], ['ELI']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'ERT ', 'ERE', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'ELA', 'ERA', 'ERT ', 'ERE', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELC', 'ELI', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'ELA', 'ERA', 'ERT ', 'ERE', 'O2', 'EMGc']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 2, [[], [], ['ELI'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'ERT ', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F4', 'C4', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'ELA', 'ERT ', 'O2', 'EMGc', 'ELC']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EOGl', 'EMGr', 'F4', 'C4', 'ELA', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 3, [[], [], ['ELI'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F3', 'F4', 'C4', 'ELA', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 4, [[], [], ['ELI'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F3', 'C4', 'ELA', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'C4', 'ERT ', 'O2']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 5, [[], [], ['ELI'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELI', 'EMGr', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'F4', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EMGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'EMGr', 'C4']], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 1.5, 6, [[], [], ['ELI'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'ELE', 'ERA', 'ERT ', 'F3', 'ERE', 'ERI', 'EOGr', 'ERB', 'EMGl']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 1, [['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ERA', 'ERT ', 'F3', 'C4', 'ERE', 'ELA', 'ERI', 'EOGr', 'EMGl', 'ELE']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'ELB', 'M1', 'ELA', 'ERI']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 1, [['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ELB', 'ELA', 'ERI']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 1, [[], [], ['EMGr'], [], [], []], ['EMGr']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'ELE', 'ERA', 'ERT ', 'F3', 'ERE']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 2, [['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ERA', 'ERT ', 'F3', 'C4', 'ERE']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA', 'ERT ', 'ERE', 'F3', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 2, [['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 2, [[], [], ['EMGr'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 3, [['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F4', 'O2', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERE', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 3, [['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'ERE', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 3, [[], [], ['EMGr'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELI', 'EOGl', 'M2', 'F4', 'C4', 'O2', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 4, [['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F4', 'O2', 'ELB', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'F4', 'C4', 'O2', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 4, [['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F4', 'C4', 'O2', 'ELB']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 4, [[], [], ['EMGr'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGl', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 5, [['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F4', 'O2', 'C4']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 5, [['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 5, [[], [], ['EMGr'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGl', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.0, 6, [['ELC', 'ELT', 'ELI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.0, 6, [['ELC', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 1.5, 6, [[], [], ['EMGr'], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_0noise_75', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'ELE', 'F3', 'C4', 'ELI', 'ERC', 'O2']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'M1', 'EMGc'], ['ELA', 'EMGr', 'M1', 'F3'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELA', 'EOGl', 'M1', 'O1', 'F4'], ['ELA', 'ELT']], ['ELA', 'EMGr', 'M1', 'O1', 'EMGc', 'ELB', 'F3', 'ELT', 'EOGr', 'EOGl', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2']], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ERE', 'EOGr', 'EMGl', 'C3', 'F4', 'ELE', 'ERB', 'F3', 'M2', 'ELI', 'ERC', 'O2']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'ERA', 'EOGr', 'EMGl', 'C3', 'F4', 'ELE', 'F3', 'M2', 'C4', 'O2', 'ELI', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'O1', 'EMGc', 'C3', 'F4', 'F3', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'M1', 'EMGc'], ['ELA', 'EMGr', 'M1', 'F3'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELA', 'EOGl', 'M1', 'O1', 'F4'], ['ELA', 'ELT']], ['ELA', 'M1', 'O1', 'EMGc', 'EOGl']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2']], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'EOGr', 'C3', 'F4', 'ERB', 'F3', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc', 'EOGr', 'EMGl', 'C3', 'F4', 'F3', 'C4', 'ELI', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'M1', 'O1', 'EMGc', 'C3', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'M1', 'EMGc'], ['ELA', 'EMGr', 'M1', 'F3'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELA', 'EOGl', 'M1', 'O1', 'F4'], ['ELA', 'ELT']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2']], ['ELA', 'ELB', 'ELT', 'EOGl', 'M1', 'O1', 'EMGc', 'EOGr', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'EOGl', 'M1', 'O1', 'EMGc', 'EOGr', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'M1', 'O1', 'EMGc', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'M1', 'EMGc'], ['ELA', 'EMGr', 'M1', 'F3'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELA', 'EOGl', 'M1', 'O1', 'F4'], ['ELA', 'ELT']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2']], ['ELA', 'ELB', 'ELT', 'O1', 'EMGc', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'O1', 'EMGc', 'F4', 'F3']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'M1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'M1', 'EMGc'], ['ELA', 'EMGr', 'M1', 'F3'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELA', 'EOGl', 'M1', 'O1', 'F4'], ['ELA', 'ELT']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2']], ['ELA', 'ELB', 'ELT', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'O1', 'EMGc', 'F4']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'M1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'M1', 'EMGc'], ['ELA', 'EMGr', 'M1', 'F3'], ['ELA', 'ELT', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELA', 'EOGl', 'M1', 'O1', 'F4'], ['ELA', 'ELT']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2']], ['ELA', 'ELT', 'O1']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'O1']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], [], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'ELC', 'ERB', 'ERE', 'EMGr', 'ELI', 'EMGl', 'ELB', 'C4', 'ERA', 'EMGc', 'M1']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'ELC', 'ERB', 'ELI', 'ERE', 'EMGl', 'ELB', 'ERA', 'EMGc', 'O2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc', 'ERE', 'ELI', 'EMGl', 'ERA', 'O2', 'M1']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], [], ['ELA'], [], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'ELT', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'ERB', 'ERE', 'ELB']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'ELC', 'ERB', 'ERE', 'ELB', 'ERA', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ERE', 'ERA', 'O2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], [], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'ERC', 'EOGl', 'O1', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERC', 'F3', 'O1', 'M2', 'ELC', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'ERE', 'O2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], [], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'F3', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ERB', 'ERC', 'F3', 'M2', 'C4', 'ERE']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], [], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'F3', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ERC', 'F3', 'M2', 'ERE']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], [], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2'], ['ELA', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'M2', 'C4', 'O2'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'ELT', 'ERC', 'F3', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], [], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'ERB', 'EOGr', 'EOGl', 'C4', 'C3', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'EMGr', 'M1', 'F3', 'M2'], ['ELA', 'ELI', 'ERB', 'M1', 'EMGc'], ['ELA', 'EOGl', 'M1']], ['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'EOGr', 'M2', 'ELI', 'ERB', 'EOGl']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc', 'ELB', 'ELI', 'ERT ', 'EOGl', 'F4', 'C4', 'ELE', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4']], ['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'C4', 'ELE', 'C3', 'O2', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], ['ELA'], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'EMGc']], ['ELA', 'ERA', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'ERB', 'EOGr']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'EMGr', 'M1', 'F3', 'M2'], ['ELA', 'ELI', 'ERB', 'M1', 'EMGc'], ['ELA', 'EOGl', 'M1']], ['ELA', 'ERA', 'M1', 'F3']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc', 'ELB', 'ELI', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4']], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ELB', 'ELI', 'C4', 'C3']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], ['ELA'], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'EMGc']], ['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'ELI', 'EOGr']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'EMGr', 'M1', 'F3', 'M2'], ['ELA', 'ELI', 'ERB', 'M1', 'EMGc'], ['ELA', 'EOGl', 'M1']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELB', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4']], ['ELA', 'EOGr', 'EMGr', 'M1', 'O1', 'EMGc', 'ELB', 'ELI', 'C4']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'EMGc']], ['ELA', 'EMGr', 'M1', 'F3', 'EMGc', 'ELB', 'ELI', 'EOGr']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'EMGr', 'M1', 'F3', 'M2'], ['ELA', 'ELI', 'ERB', 'M1', 'EMGc'], ['ELA', 'EOGl', 'M1']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'EMGr', 'M1', 'F3', 'O1', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4']], ['ELA', 'EMGr', 'M1', 'O1', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'EMGc']], ['ELA', 'EMGr', 'M1', 'F3', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'EMGr', 'M1', 'F3', 'M2'], ['ELA', 'ELI', 'ERB', 'M1', 'EMGc'], ['ELA', 'EOGl', 'M1']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'EMGr', 'M1', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4']], ['ELA', 'EMGr', 'M1', 'O1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'EMGc']], ['ELA', 'EMGr', 'M1', 'F3', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ERA', 'EMGr', 'M1', 'F3', 'O1'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'O1'], ['ELA', 'EMGr', 'M1', 'F3', 'M2'], ['ELA', 'ELI', 'ERB', 'M1', 'EMGc'], ['ELA', 'EOGl', 'M1']], ['ELA', 'M1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4']], ['ELA', 'EMGr', 'M1', 'O1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4']], ['ELA', 'EMGr', 'M1', 'O1']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ELB', 'EOGr', 'EOGl', 'ELC', 'ERB', 'ERC', 'F4', 'ELE']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'ELB', 'ERB', 'ERC', 'EOGl', 'O2', 'ELC', 'EMGc', 'EMGl', 'C3', 'ELE']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ERI', 'EOGl', 'EMGc', 'EMGl', 'EMGr', 'C3', 'ELE']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'EOGl', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'ELB', 'ERB', 'ERC', 'EOGl', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'ERI', 'EOGl', 'EMGc', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'C4', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'ERE', 'F3', 'O1', 'M2', 'C4', 'ELB', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ERE', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'F3', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELI', 'ERB', 'F3', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELI', 'ERB', 'O1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELI', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc']], ['ELC', 'ELI', 'ERB', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'ERI', 'EOGr', 'EMGr', 'O1', 'EMGc', 'ERB', 'C4', 'ELI', 'ERC', 'EMGl']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'EOGl', 'C3', 'F4'], ['ELA', 'ELB', 'EOGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'EOGl', 'M1'], ['ELA', 'ELB', 'EMGr'], ['ELA', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'O1']], ['ELA', 'ELB', 'EOGl', 'C3', 'F4', 'EOGr', 'M1', 'O1', 'EMGc', 'EMGr', 'C4', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ERI', 'EOGr', 'EMGr', 'C4', 'ERB', 'ELI', 'EMGl', 'ERA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELI', 'ERC', 'EMGr', 'C4', 'O2', 'ERB', 'ERI', 'M1', 'EOGr', 'ERE', 'EMGl']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 1, [['ELA'], [], [], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 1, [['ELA'], [], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'M1', 'M2', 'F4', 'EOGr', 'EMGr', 'O1', 'EMGc', 'ERB', 'ELI', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'EOGl', 'C3', 'F4'], ['ELA', 'ELB', 'EOGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'EOGl', 'M1'], ['ELA', 'ELB', 'EMGr'], ['ELA', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'O1']], ['ELA', 'ELB', 'F4', 'M1', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'EOGr', 'EMGr', 'C4', 'ERB', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELI', 'ERC', 'EMGr', 'C4', 'M1', 'EOGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 2, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 2, [['ELA'], [], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'M1', 'M2', 'F4', 'EMGr', 'O1', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'EOGl', 'C3', 'F4'], ['ELA', 'ELB', 'EOGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'EOGl', 'M1'], ['ELA', 'ELB', 'EMGr'], ['ELA', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'O1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'EOGr', 'EMGr', 'C4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'EMGr', 'M1', 'EOGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 3, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 3, [['ELA'], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'M2', 'F4', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'EOGl', 'C3', 'F4'], ['ELA', 'ELB', 'EOGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'EOGl', 'M1'], ['ELA', 'ELB', 'EMGr'], ['ELA', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'O1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'M2', 'F4', 'EMGr', 'C4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'C3', 'O1', 'M2', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 4, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 4, [['ELA'], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'F4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'EOGl', 'C3', 'F4'], ['ELA', 'ELB', 'EOGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'EOGl', 'M1'], ['ELA', 'ELB', 'EMGr'], ['ELA', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'O1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M2', 'F4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'O1', 'M2', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 5, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 5, [['ELA'], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'F4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'EOGl', 'C3', 'F4'], ['ELA', 'ELB', 'EOGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'EOGl', 'M1'], ['ELA', 'ELB', 'EMGr'], ['ELA', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'O1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M2', 'F4']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EOGl', 'O1', 'M2']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 1.5, 6, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'mm', 5.0, 6, [['ELA'], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 2000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc', 'ERB', 'F3', 'O1', 'EOGr', 'F4', 'C4', 'O2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'O2', 'F4', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 1, [[], [], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 1, [[], [], ['M2'], [], [], []], ['M2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'M2', 'EMGc', 'ERB', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc', 'ERB', 'F3']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ERB', 'EOGr']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 2, [[], [], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 2, [[], [], ['M2'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELT', 'EOGl', 'M2', 'EMGc', 'O1']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELA', 'ELC', 'ELT', 'ELI', 'EOGl', 'O1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELC', 'ELT', 'ELI', 'ERC', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 3, [[], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 3, [[], [], ['M2'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELT', 'M2', 'EMGc', 'O1']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELA', 'ELC', 'ELT', 'O1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELC', 'ELT', 'ERC', 'O1', 'M2', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 4, [[], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 4, [[], [], ['M2'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'M2', 'O1']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELA', 'O1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERC', 'O1', 'M2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 5, [[], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 5, [[], [], ['M2'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'M2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EOGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELT', 'ERA', 'EOGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'M2', 'F4', 'C4']], ['ELA', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['O1', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 1.5, 6, [[], [], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 1.5, 6, [[], [], ['M2'], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 2000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'C4', 'ELE', 'EMGr', 'C3', 'O2', 'ELC', 'M2']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 1, [['ELA', 'M1', 'F3', 'F4'], ['ELA', 'ERA', 'ERB', 'O1', 'EMGc'], ['ELA', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERB', 'F4', 'C4', 'EMGc'], ['ELA', 'EMGr', 'M1']], ['ELA', 'M1', 'F3', 'F4', 'ERA', 'ERB', 'O1', 'EMGc', 'C3', 'ELB', 'C4', 'EMGr']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERB', 'ERE', 'M2', 'C4', 'ELE', 'C3']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERB', 'ERE', 'C3', 'M2', 'C4', 'ELE', 'ELC', 'ERC']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'M1', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERB', 'C4', 'EMGr', 'C3']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 2, [['ELA', 'M1', 'F3', 'F4'], ['ELA', 'ERA', 'ERB', 'O1', 'EMGc'], ['ELA', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERB', 'F4', 'C4', 'EMGc'], ['ELA', 'EMGr', 'M1']], ['ELA', 'F4', 'ELB']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EMGr', 'M1', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERB', 'ERE', 'C4']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERE', 'C4']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc']], ['ELA', 'M1', 'O1', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 3, [['ELA', 'M1', 'F3', 'F4'], ['ELA', 'ERA', 'ERB', 'O1', 'EMGc'], ['ELA', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERB', 'F4', 'C4', 'EMGc'], ['ELA', 'EMGr', 'M1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'EMGr', 'M1', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERE']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'EMGr', 'M1', 'O1', 'F4', 'EMGc', 'ELT', 'ELI', 'ERE', 'C4']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc']], ['ELA', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 4, [['ELA', 'M1', 'F3', 'F4'], ['ELA', 'ERA', 'ERB', 'O1', 'EMGc'], ['ELA', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERB', 'F4', 'C4', 'EMGc'], ['ELA', 'EMGr', 'M1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'EMGr', 'M1', 'O1', 'F4', 'EMGc', 'ELT', 'ERE']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1', 'O1', 'EMGc', 'ELT', 'C4']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc']], ['ELA', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 5, [['ELA', 'M1', 'F3', 'F4'], ['ELA', 'ERA', 'ERB', 'O1', 'EMGc'], ['ELA', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERB', 'F4', 'C4', 'EMGc'], ['ELA', 'EMGr', 'M1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'O1', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'C3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc']], ['ELA', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.0, 6, [['ELA', 'M1', 'F3', 'F4'], ['ELA', 'ERA', 'ERB', 'O1', 'EMGc'], ['ELA', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERB', 'F4', 'C4', 'EMGc'], ['ELA', 'EMGr', 'M1']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'F4', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'M1', 'EMGc']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_100', 3000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2'], ['ELA', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2', 'ERC', 'ERI', 'EOGl', 'O1', 'EMGc', 'ELC', 'ERE', 'C4', 'O2', 'ELT', 'ELI', 'F4', 'EOGr']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'O1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'ELB', 'ERA', 'ERI', 'EOGr', 'EOGl', 'O1', 'C4', 'EMGc', 'ELC', 'ELT', 'ELI', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 1, [['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc', 'ELI', 'EMGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2'], ['ELA', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ERB', 'F3', 'M2', 'ERC', 'EOGl', 'ELC', 'ERE', 'F4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'O1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'F3', 'M2', 'ERA', 'EOGr', 'EOGl', 'O1', 'C4', 'EMGc', 'ELC', 'ELT']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 2, [['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'F3', 'M2', 'ELT', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2'], ['ELA', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc']], ['ELA', 'F3', 'M2', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'O1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ERC', 'F3', 'M2', 'ERA', 'EOGr', 'O1', 'C4', 'ELC']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 3, [['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'F3', 'M2', 'ELT', 'EOGr', 'EOGl', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2'], ['ELA', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc']], ['ELA', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'O1', 'M2', 'EMGc']], ['ELA', 'ERC', 'F3', 'M2', 'EOGr', 'C4', 'ELC']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 4, [['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERC', 'F3', 'M2', 'EOGr', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2'], ['ELA', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc']], ['ELA', 'EOGl']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'O1', 'M2', 'EMGc']], ['ELA', 'F3', 'M2', 'EOGr']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 5, [['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERC', 'F3', 'M2', 'EOGr', 'C4']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELE', 'ERB', 'EMGr', 'F3', 'C3', 'M2'], ['ELA', 'ELE', 'ERB', 'ERC', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERE', 'EOGl', 'O1', 'M2', 'F4'], ['ELA', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'EMGc']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELE', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'M2', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELC', 'ELE', 'ERB', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'O1', 'M2', 'EMGc']], ['ELA', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.0, 6, [['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'M2', 'C4'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERC', 'F3', 'M2']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_1000', 3000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'EOGl', 'F4', 'EMGc', 'ERI', 'ELA', 'ELB', 'ERA', 'ERC', 'F3']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 1, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELT', 'ELI', 'EOGl', 'F4', 'ERI', 'ELA', 'ELB', 'ERA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 1, [['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'EOGl', 'F4', 'EMGc', 'ELA', 'ELB', 'ERA', 'EOGr', 'F3']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 1, [['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2', 'ELT', 'ELE', 'ELI', 'EOGl', 'F3', 'F4', 'EMGc', 'ELA', 'ELB', 'ERA', 'EOGr', 'ERC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'ELT', 'ELI', 'EOGl', 'F4', 'EMGc', 'ELA', 'ELB', 'ERC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 2, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'C4', 'EMGc', 'ELT', 'ELI', 'EOGl', 'ELA', 'ELB']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 2, [['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'C4', 'O2', 'ELT', 'ELI', 'F4', 'EMGc', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 2, [['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'M2', 'C4', 'O2', 'ELI', 'EOGl', 'F4', 'EMGc', 'ELA', 'ERC']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERE', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 3, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 3, [['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'EMGl', 'M2', 'O2', 'EMGc', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 3, [['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERE', 'ERI', 'EMGl', 'M2', 'O2', 'EMGc', 'ELA']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'ERE', 'EMGl', 'M1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 4, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ELC', 'ERB', 'EMGl', 'M1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 4, [['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2', 'O2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 4, [['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERE', 'M2', 'O2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ERB', 'ERE', 'M1', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 5, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ERB', 'M1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 5, [['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 5, [['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ERB', 'ERE', 'M1', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.0, 6, [['ELC', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'EMGc']], ['ERB', 'M1', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.0, 6, [['ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELC', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.0, 6, [['ELC', 'ERB', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'ERE']], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc', 'ERI', 'EMGr', 'F3', 'C4', 'EOGl', 'O1', 'M2', 'F4', 'O2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2']], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc', 'ELA', 'ERI', 'EMGr', 'F3', 'M2', 'C4', 'EOGl', 'O2', 'O1', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'M2']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'ELC', 'ELT', 'ERI', 'EMGl', 'F3', 'C4', 'EOGl', 'O2', 'EMGc', 'EOGr', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'ELC', 'ELT', 'ERI', 'EMGl', 'F3', 'O1', 'O2', 'EOGl', 'EMGc', 'EOGr', 'F4', 'C4', 'M2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], ['ERE']], ['ERE']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc', 'EMGr', 'F3', 'C4', 'O1', 'F4', 'O2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2']], ['ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc', 'EMGr', 'F3', 'C4', 'O2', 'F4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'M2']], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'ELT', 'EMGl', 'F3', 'C4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2']], ['ELA', 'ELI', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'ELT', 'ERI', 'EMGl', 'F3', 'O2', 'EOGr']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ELT', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'EMGc', 'C4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2']], ['ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'EMGc', 'C4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'M2']], ['ERB', 'ERC', 'ERE', 'M1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2']], ['ERB', 'ERC', 'ERE', 'M1', 'ELT', 'O2']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERB', 'ERC', 'ERE', 'M1', 'C4']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2']], ['ERB', 'ERC', 'ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'M2']], ['ERB', 'ERC', 'ERE', 'M1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2']], ['ERB', 'ERC', 'ERE', 'M1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'M2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2']], ['ERE', 'M1', 'ELT']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'O2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'EMGc'], ['ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'M1', 'F3', 'M2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'C3'], ['ELA', 'ELC', 'ELT', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGr', 'M1', 'O2'], ['ELA', 'ELT', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'F4', 'C4'], ['ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2']], ['ERE', 'M1']], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], ['ERE']], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_10', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'ELE', 'ERA', 'ERB', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'EOGr', 'ELC', 'ELB', 'ERC', 'C3']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'ELE', 'ELI', 'ERA', 'ERB', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'ELB', 'ERC']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ELE', 'ERA', 'ERB', 'O2', 'ELB', 'ERC', 'C3']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'ERB', 'O2', 'EMGc', 'ERA', 'F4', 'ELB', 'ERC', 'C3']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 1, [[], ['ELA'], ['ELA'], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 1, [[], ['ELA'], [], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'ELE', 'ERB', 'C4', 'EMGc', 'ELI', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'ELI', 'ERB', 'F4', 'C4', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc', 'ERB', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELT', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'ERB', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 2, [[], ['ELA'], ['ELA'], [], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 2, [[], ['ELA'], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'ERB', 'C4', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'F3', 'O1', 'ELI', 'ERB', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELT', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELI', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 3, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 3, [[], ['ELA'], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'M1', 'O1', 'ERB', 'C4', 'EMGc', 'ELI']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'ELI', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'ELT', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 4, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 4, [[], ['ELA'], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'O1', 'ERB', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'ELI', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'EOGl', 'F3', 'O1', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 5, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 5, [[], ['ELA'], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'O1']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELT', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc']], ['ELA', 'EOGl', 'EMGr']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'EMGc']], ['ELA', 'EOGl', 'EMGr', 'O1', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'EOGl', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 1.5, 6, [[], ['ELA'], ['ELA'], [], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 1.5, 6, [[], ['ELA'], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'F3', 'EMGc', 'ELC', 'ERA', 'EOGr', 'ERT ']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'F3', 'EMGc', 'ELC', 'EOGr', 'ELB', 'O2']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'F3', 'EMGc', 'ERA', 'EOGr', 'ERT ', 'O2']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'ERB', 'ERI', 'F3', 'EMGc', 'ELB', 'ELE', 'EOGr', 'ERT ', 'F4', 'O2']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], [], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 1, [['ELA'], [], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'F3', 'EMGc', 'ERA', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'F3', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'F3', 'EMGc', 'ERA', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'ERB', 'F3', 'EMGc', 'EOGr']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], [], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 2, [['ELA'], [], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ELT', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'C4', 'ERB', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'EMGr', 'M1', 'O1', 'C4', 'ERB', 'EMGc', 'ERA']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'C4', 'ERB', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], [], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 3, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'O1', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'EOGl', 'EMGr', 'O1', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELI', 'ERE', 'EOGl', 'O1', 'C4', 'EMGc', 'ERA']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'ELI', 'ERE', 'EOGl', 'C4', 'EMGc']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], [], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 4, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'EOGl', 'EMGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELI', 'ERE', 'EOGl', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'EOGl', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], [], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 5, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4']], ['ELA', 'ERE', 'EOGl', 'EMGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'EMGc'], ['ELA', 'ELT', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'EOGl', 'EMGr', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELI', 'ERE', 'EOGl', 'O1', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4'], ['ELA', 'ELT', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERE', 'EOGr', 'EOGl', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELA', 'EOGl', 'C4']], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], [], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 1.5, 6, [['ELA'], [], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_30', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'ELC', 'ELI', 'EMGl', 'O1', 'O2', 'ERA', 'ERC', 'F4', 'EMGr', 'EMGc', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ERT ', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ERT ', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'ELC', 'O1', 'O2', 'ERA', 'F4', 'ELT', 'EMGc', 'ERE', 'EMGl']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'ELC', 'ELI', 'O1', 'EMGc', 'ERA', 'M2', 'F4', 'ERC', 'EMGl', 'O2', 'ERE']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'ELI', 'O1', 'EMGc', 'ERA', 'M2', 'F4', 'O2', 'ERE']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'M1', 'F3', 'O1', 'O2', 'ERA', 'ERC']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ERT ', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ERT ', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ERT ', 'F3', 'O1', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'M1', 'F3', 'ELC', 'O1', 'EMGc', 'ERA', 'M2', 'EMGl', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'M1', 'F3', 'EMGc', 'ERA', 'M2', 'O2']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ERT ', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ERT ', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'O2']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'F3', 'ELC', 'O1']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERI', 'M1', 'F3', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2']], ['ELA', 'ELT', 'ELE', 'M1', 'F3', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ERT ', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ERT ', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'O2']], ['ELA', 'ELT', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ERT ', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ERT ', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'O2']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EMGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'M1', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'O2']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELB', 'ELC', 'ERT ', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ERT ', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'EOGl', 'F3', 'O1', 'O2'], ['ELA', 'ELB', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'O2']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3'], ['ELA', 'ELC', 'ELI', 'ERT ', 'ERI', 'F3', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'F3', 'M2', 'F4'], ['ELA', 'ELC', 'ELT', 'ERA', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], [], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc', 'ELB', 'ELT', 'ELE', 'ERC', 'ERT ', 'F3', 'C3', 'ELI', 'ERE', 'O1', 'EMGl', 'F4', 'C4']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EMGl', 'F3', 'O1'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc', 'ELB', 'ELC', 'ELT', 'ERC', 'ERT ', 'C3', 'ELI', 'F3', 'EMGl', 'O1', 'M2', 'EOGl']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'EMGc']], ['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc', 'ELB', 'ELC', 'ELT', 'ERC', 'ERT ', 'F3', 'C3', 'O1', 'ELI', 'ERE', 'F4', 'M2', 'C4']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc', 'ELB', 'ELC', 'ELT', 'ERC', 'ERT ', 'ERE', 'O1', 'M2', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], ['ELA'], [], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 1, [['ELA'], [], [], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 1, [[], [], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERI', 'M1', 'O2', 'ELB', 'ELT', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EMGl', 'F3', 'O1'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ELE', 'ERA', 'ERI', 'ELC', 'ELT', 'ERT ', 'F3', 'EMGl', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI', 'EOGl', 'M1', 'ELB', 'ELC', 'ELT', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'F3', 'EMGc', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], ['ELA'], [], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 2, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 2, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERI', 'O2', 'ELB', 'ELT', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EMGl', 'F3', 'O1'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI', 'EOGl', 'M1', 'ELB', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'EMGc', 'ELB']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 3, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 3, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERI', 'O2', 'ELT', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EMGl', 'F3', 'O1'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERI', 'F3', 'EMGc']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 4, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 4, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EMGl', 'F3', 'O1'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI', 'ERT ', 'F3', 'O1']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 5, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 5, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERI']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELE', 'ERA', 'ERI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'C3'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'F3', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'EMGl', 'F3', 'O1'], ['ELA', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERI', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'F3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ERA', 'ERT ', 'ERI', 'EOGl', 'M1', 'F3', 'O1', 'EMGc']], ['ELA', 'ERA', 'ERI']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'F3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERI', 'EOGl', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EOGl', 'F3', 'O1', 'EMGc']], ['ELA', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], ['ELA'], [], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.0, 6, [['ELA'], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 3.0, 6, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_50', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'ELC', 'ERB', 'C4', 'EOGr', 'O1', 'M1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'C4'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'C4'], ['ELA', 'ELE', 'ERA', 'ERB', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'C3', 'O2'], ['ELA', 'ELT', 'ERC', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2', 'ERA', 'C4', 'ELI', 'ERB', 'ERE', 'C3', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2', 'ELC', 'ERB', 'M1', 'C4', 'F3', 'O1', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2', 'ELC', 'ERB', 'M1', 'C4', 'F3', 'O1', 'ERE', 'M2']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 1, [[], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERA'], ['ELA']], ['ELA', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'C3', 'O2', 'ELC', 'ERB', 'C4', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'C4'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'C4'], ['ELA', 'ELE', 'ERA', 'ERB', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'C3', 'O2'], ['ELA', 'ELT', 'ERC', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ERC', 'O2', 'ERA', 'C4', 'ERB']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'C3', 'O2', 'ERB', 'C4', 'O1', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'C3', 'O2', 'ERB', 'C4', 'F3', 'O1', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 2, [[], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'O2', 'ERB', 'C4', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'C4'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'C4'], ['ELA', 'ELE', 'ERA', 'ERB', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'C3', 'O2'], ['ELA', 'ELT', 'ERC', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ERC', 'O2']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'O2', 'ERB', 'C4', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'O2']], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'C3', 'O2', 'ERB', 'C4', 'F3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 3, [[], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERB', 'O1']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'C4'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'C4'], ['ELA', 'ELE', 'ERA', 'ERB', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'C3', 'O2'], ['ELA', 'ELT', 'ERC', 'O1', 'O2']], ['ELA', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ERA', 'ERT ']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'O2']], ['ELA', 'ELE', 'ELI', 'C3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 4, [[], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELE', 'ERA', 'ERB']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'C4'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'C4'], ['ELA', 'ELE', 'ERA', 'ERB', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'C3', 'O2'], ['ELA', 'ELT', 'ERC', 'O1', 'O2']], ['ELA', 'ELE']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'O2']], ['ELA', 'ELE', 'C3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 5, [[], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'M1', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ERC', 'F4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'C4'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'C4'], ['ELA', 'ELE', 'ERA', 'ERB', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'C3', 'O2'], ['ELA', 'ELT', 'ERC', 'O1', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F4', 'C4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERT ', 'F3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERC', 'ERE', 'C3', 'O1', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ELE', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'M2', 'O2']], ['ELA', 'ELE', 'C3']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 1.5, 6, [[], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4', 'ELE', 'ELI', 'M1', 'C3', 'O2', 'ERT ', 'EMGl', 'ERE', 'EOGl', 'F3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1'], ['ELA', 'ERA', 'M1', 'C3', 'O1', 'C4'], ['ELA', 'ELE', 'ERT ', 'ERE', 'F4'], ['ELA', 'ELC', 'ERE', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'O2']], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'C3', 'M1', 'C4', 'ELE', 'ERT ', 'ERE', 'F4', 'ELC', 'O2', 'ELT', 'ELI']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'M1', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'O1', 'O2']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4', 'ELE', 'EMGr', 'M1', 'C3', 'O2', 'F4', 'ERE', 'EOGl', 'ERI']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'ELE', 'ELI', 'EMGr', 'M1', 'O2', 'ERE', 'ELT', 'EOGl', 'ERI', 'F3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4', 'ELI', 'M1', 'C3', 'O2', 'ERT ', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1'], ['ELA', 'ERA', 'M1', 'C3', 'O1', 'C4'], ['ELA', 'ELE', 'ERT ', 'ERE', 'F4'], ['ELA', 'ELC', 'ERE', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'O2']], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C3', 'ERE', 'ELC', 'O2']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'M1', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'O1', 'O2']], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'C4', 'M1', 'C3', 'O2', 'F4', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'O1', 'F4', 'C4', 'ELI', 'M1', 'O2', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'ERA', 'EOGr', 'O1', 'C4', 'C3', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1'], ['ELA', 'ERA', 'M1', 'C3', 'O1', 'C4'], ['ELA', 'ELE', 'ERT ', 'ERE', 'F4'], ['ELA', 'ELC', 'ERE', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'O2']], ['ELA', 'ERA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'M1', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'O1', 'O2']], ['ELA', 'ELT', 'ERA', 'ERT ', 'EOGr', 'O1', 'C4', 'C3', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'EOGr', 'C3', 'O1', 'C4', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'EOGr', 'O1', 'C3']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1'], ['ELA', 'ERA', 'M1', 'C3', 'O1', 'C4'], ['ELA', 'ELE', 'ERT ', 'ERE', 'F4'], ['ELA', 'ELC', 'ERE', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'M1', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'O1', 'O2']], ['ELA', 'ERA', 'EOGr', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'EOGr', 'C3', 'O1', 'ERE']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1'], ['ELA', 'ERA', 'M1', 'C3', 'O1', 'C4'], ['ELA', 'ELE', 'ERT ', 'ERE', 'F4'], ['ELA', 'ELC', 'ERE', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'M1', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'O1', 'O2']], ['ELA', 'ERA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERE', 'M1', 'F3', 'O1', 'O2']], ['ELA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'O1', 'M2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1'], ['ELA', 'ERA', 'M1', 'C3', 'O1', 'C4'], ['ELA', 'ELE', 'ERT ', 'ERE', 'F4'], ['ELA', 'ELC', 'ERE', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'O2']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'O1', 'M2', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERT ', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'M1', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'O1', 'O2']], ['ELA', 'ERA', 'O1']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'F4'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O2']], ['ELA', 'ERA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], [], ['ELA']], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_1noise_75', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'ELC', 'ELT', 'EMGr', 'M1', 'O2', 'EMGc', 'ELI', 'C3', 'ERB', 'C4', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELE', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'EOGl'], ['ELA', 'ELB', 'ERA', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGc'], ['ELA', 'ELI', 'EOGl'], ['ELA', 'ELE', 'EOGl']], ['ELA', 'ELE', 'EOGr', 'EOGl', 'ELB', 'ERA', 'ELC', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc', 'ELT', 'EMGr', 'C3', 'ELI', 'O1', 'ELC', 'ERB', 'ERE', 'M1', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc', 'ERC', 'EMGr', 'M1', 'F3', 'ELI', 'ERE', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 1, [[], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'F4', 'ELC', 'EMGr', 'O2', 'EMGc', 'ELI', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELE', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'EOGl'], ['ELA', 'ELB', 'ERA', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGc'], ['ELA', 'ELI', 'EOGl'], ['ELA', 'ELE', 'EOGl']], ['ELA', 'ELE', 'EOGl', 'ELB', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc', 'EMGr', 'C3', 'ELI', 'O1', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc', 'ERC', 'EMGr', 'M1', 'ERE', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 2, [[], ['ELA'], ['ELA'], [], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGr', 'EOGl', 'ELC', 'EMGr', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELE', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'EOGl'], ['ELA', 'ELB', 'ERA', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGc'], ['ELA', 'ELI', 'EOGl'], ['ELA', 'ELE', 'EOGl']], ['ELA', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'O2', 'EMGc', 'EMGr', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'EOGl', 'C3', 'F4', 'O2', 'EMGc', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 3, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELE', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'EOGl'], ['ELA', 'ELB', 'ERA', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGc'], ['ELA', 'ELI', 'EOGl'], ['ELA', 'ELE', 'EOGl']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGl', 'F4', 'O2', 'EMGc', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 4, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C4', 'EMGc']], ['ELA', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELE', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'EOGl'], ['ELA', 'ELB', 'ERA', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGc'], ['ELA', 'ELI', 'EOGl'], ['ELA', 'ELE', 'EOGl']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'ERA', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGl', 'O2', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 5, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELA', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'F3', 'C4', 'EMGc']], ['ELA', 'ELE', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELE', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'EOGl'], ['ELA', 'ELB', 'ERA', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGc'], ['ELA', 'ELI', 'EOGl'], ['ELA', 'ELE', 'EOGl']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELE', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'EOGl', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 1.5, 6, [[], ['ELA'], ['ELA'], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2', 'ERA', 'ERC', 'M1', 'O1', 'EMGc', 'ELT', 'F4', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERC'], ['ELA', 'ELB']], ['ELA', 'ELB', 'O2', 'ERA', 'ERE', 'O1', 'C4', 'ERT ', 'EMGc', 'ELC', 'ERC']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2', 'ELI', 'ERA', 'ERE', 'O1', 'M2', 'EMGc', 'EMGl', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'C3', 'EMGc', 'ELT', 'EMGl', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'F3', 'C4', 'ERA', 'O1', 'EMGc', 'ELT']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERC'], ['ELA', 'ELB']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'ELI', 'O1', 'M2', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'ELC', 'EMGc', 'ELT']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERC'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'EOGl', 'C4', 'O1']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERC', 'ERT ', 'EOGl', 'C4', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERC'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ', 'EOGl', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERC'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGl', 'EMGr', 'F3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'ERE', 'EOGl', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'O2'], ['ELA', 'ELB', 'ERA', 'ERE', 'O1', 'C4'], ['ELA', 'ELB', 'ERT ', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERC'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERE', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'C3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERI', 'EOGl', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], [], ['ELA'], ['ELA']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'O2']], ['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELT', 'ELE', 'EOGr', 'M1', 'ELC', 'ERA', 'ERC', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ELB', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'EMGl'], ['ELA', 'ELB', 'EOGr', 'EMGl']], ['ELA', 'ELB', 'C3', 'ERA', 'EOGr', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2', 'EOGr', 'EMGr', 'M1', 'ELC', 'ERA', 'ERC', 'F3', 'F4', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2', 'EMGr', 'M1', 'F4', 'ELC', 'ERA', 'EOGr', 'F3', 'M2', 'ERC', 'EOGl', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'O2']], ['ELA', 'ELB', 'EMGl', 'C3', 'F4', 'O2', 'EOGr', 'M1', 'ELC', 'ERA', 'ERC']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ELB', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'EMGl'], ['ELA', 'ELB', 'EOGr', 'EMGl']], ['ELA', 'ELB', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'EMGl', 'C3', 'O2', 'EOGr', 'EMGr', 'M1', 'ERA', 'ERC', 'F4', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'ELE', 'EMGl', 'C3', 'C4', 'M1', 'ERA', 'EOGr', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'O2']], ['ELA', 'ELB', 'EMGl', 'C3', 'O2', 'EOGr', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ELB', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'EMGl'], ['ELA', 'ELB', 'EOGr', 'EMGl']], ['ELA', 'ELB', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'C3', 'EOGr', 'M1', 'ERA', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'C3', 'ERA', 'EOGr', 'EOGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'O2']], ['ELA', 'ELB', 'EMGl', 'O2', 'EOGr', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ELB', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'EMGl'], ['ELA', 'ELB', 'EOGr', 'EMGl']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'EOGr', 'M1', 'ERA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'ERA', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'O2']], ['ELA', 'ELB', 'EMGl', 'EOGr', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ELB', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'EMGl'], ['ELA', 'ELB', 'EOGr', 'EMGl']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'EOGr', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERE', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'EOGr', 'EMGl', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'O2']], ['ELA', 'ELB', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ELB', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'EMGl'], ['ELA', 'ELB', 'EOGr', 'EMGl']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'M1', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERE', 'ERI', 'EMGl', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'EMGr', 'M1', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'M2'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'F4']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.0, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr', 'EMGl', 'F4', 'M2', 'ERB', 'ERC', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB', 'EMGc', 'C3']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'C3', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr', 'ERE', 'EOGl', 'O2', 'ELE', 'ERB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc', 'EOGr', 'ERE', 'O1', 'EOGl', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERI', 'C3', 'C4', 'EMGc', 'ELC', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'C3', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc', 'ELC', 'EOGr', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'EMGc', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'C3', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'C3', 'EMGc', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'C3', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'C3', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'C4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT ', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERI', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'F3', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB', 'C3'], ['ELA', 'ELB', 'EMGc'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'C3', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT ', 'ERI', 'C3', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERI', 'C3', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 2.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 3.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C4'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc', 'ELT', 'ERT ', 'EOGr', 'M1', 'C4', 'ERA', 'M2', 'F4', 'ELC', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc', 'ELT', 'ERA', 'ERT ', 'EOGr', 'M1', 'ELC', 'M2', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc', 'ERT ', 'EOGr', 'M1', 'M2', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], ['ELA'], [], [], [], []], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C4'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'ERT ', 'C4', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'ERA', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'ERT ', 'EOGr', 'M1', 'M2', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], ['ELA'], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C4'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ERC', 'EOGl', 'EMGl', 'ERT ', 'C4', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'ERA', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], ['ELA'], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C4'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ERC', 'EMGl', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERC', 'EMGl', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], ['ELA'], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C4'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ERC', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'EMGl', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], ['ELA'], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C4'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGl', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'C4'], ['ELA', 'ELT', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'ERC', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'F4', 'C4', 'O2']], ['ELA', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'EMGc'], ['ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C4'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4'], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2']], ['ERC', 'EMGl', 'F3', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], ['ELA'], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_10noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc', 'ELC', 'C3', 'O2', 'ERC', 'ERT ', 'EOGr', 'ERB', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc', 'C3', 'O2', 'ERC', 'ERT ', 'EOGr', 'ELI', 'M2', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'O2'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2']], ['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'ELT', 'ERT ', 'F4', 'ERC', 'F3', 'EOGr', 'ELI', 'M2']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 1, [[], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGr', 'O1', 'EMGc', 'ELC', 'O2', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EMGr', 'M1', 'O1', 'EMGc', 'C3', 'O2', 'ERT ', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'O2'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2']], ['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc', 'ELC', 'ELT', 'ERT ', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 2, [[], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'EMGr', 'EMGc', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'ERE', 'EMGr', 'M1', 'EMGc', 'C3']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'O2'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2']], ['ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc', 'ELT', 'ERT ']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 3, [[], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'O2'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2']], ['ELB', 'ELE', 'ERA', 'EMGl', 'EMGr', 'O1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 4, [[], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'ERA', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELC', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'O2'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2']], ['ELB', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 5, [[], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERA', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'EOGr', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'ELE', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O2']], ['ELA', 'ELB', 'EMGr']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELE', 'ERA', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'O2'], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2']], ['ELB', 'ELE', 'EMGr', 'O1', 'O2']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 1.5, 6, [[], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERI', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4', 'EOGl', 'O1', 'C4', 'EMGc', 'ERI', 'EMGl', 'C3', 'M2', 'O2', 'ELI', 'EOGr']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4', 'ERT ', 'EOGl', 'O1', 'EMGc', 'ELI', 'EMGl', 'M2', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'ERT ', 'EOGl', 'EMGc', 'ERA', 'ERC', 'EMGl', 'M2', 'F3']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 1, [['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB', 'ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 1, [['ELB'], [], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERI', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERT ', 'M1', 'F4', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'M1', 'C3', 'ERT ', 'O1', 'EMGc', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'ERT ', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 2, [['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB', 'ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 2, [['ELB'], [], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERI', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERB', 'M1', 'EMGc', 'ELE']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ERB', 'EOGr', 'M1', 'O1', 'EOGl', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 3, [['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 3, [['ELB'], [], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERI', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERB', 'M1', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ERB', 'EOGr', 'M1', 'O1', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 4, [['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 4, [['ELB'], [], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERI', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'M1', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ELC', 'ERB', 'EOGr', 'M1', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 5, [['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 5, [['ELB'], [], ['ELB'], ['ELB'], ['ELB'], ['ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EMGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGl', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERI', 'EMGl', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'M1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERT ', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERI', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'M1', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'F4', 'EMGc']], ['ELA', 'ELB', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ELT', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ERB', 'ERI', 'EOGr', 'EOGl', 'M1', 'O1', 'F4', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'EMGc']], ['ELB', 'ERB', 'EOGr', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 1.5, 6, [['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELB'], ['ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.0, 6, [['ELB'], [], ['ELB'], ['ELB'], ['ELB'], ['ELB']], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_50noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGl', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ERT ', 'M1']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3', 'ERI', 'F3', 'C4', 'ELC', 'EOGr', 'EMGr', 'M2', 'EMGc', 'O1', 'EMGl', 'ERT ', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 1, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3', 'ELC', 'EMGr', 'C4', 'EOGr', 'M2', 'EMGc', 'ELI', 'O1', 'ERC', 'EMGl', 'M1']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 1, [['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'ELC', 'EMGl', 'EMGr', 'M2', 'C4', 'ELT', 'EOGr', 'EMGc', 'ELI', 'F4']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGl', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ERT ', 'M1']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGl', 'C4', 'ELC', 'O1']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 2, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3']], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'ELC', 'EMGr', 'C4', 'EMGc', 'ELI', 'EMGl']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 2, [['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'ELC', 'EMGl', 'EMGr', 'M2', 'C4', 'EMGc', 'ELI']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGl', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ERT ', 'M1']], ['ELA', 'ELB', 'ERE', 'C4']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 3, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3']], ['ELA', 'ELB', 'ELE', 'ERT ', 'ERI', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 3, [['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELB', 'ELE', 'ERT ', 'ERI', 'ELC', 'EMGc']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGl', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ERT ', 'M1']], ['ELA', 'ELB', 'ERE']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 4, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3']], ['ELA', 'ELB', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 4, [['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELB', 'ERI', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGl', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ERT ', 'M1']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 5, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 5, [['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELB', 'ERI', 'ELC']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERE', 'EOGl', 'F3', 'O1', 'C4'], ['ELA', 'ELB', 'ERI', 'EOGl', 'EMGl', 'O1'], ['ELA', 'ELB', 'ERT ', 'M1']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.0, 6, [['ELA'], ['ELA'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERA', 'ERB', 'ERT ', 'ERE', 'ERI', 'EOGl', 'F3', 'C3'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGr', 'F3', 'C4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ELA', 'ELB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.0, 6, [['ELB', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1'], ['ELB', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'C4'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ELC', 'ERB', 'ERT ', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 1.5, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB'], ['ELB']], ['ELB']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['Combinations\\EEG_manual_100-25cha_noise_75noise_1000', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C3', 'C4', 'ELT', 'ERI', 'EMGr', 'ERB']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ELI', 'EOGl', 'M1', 'C3', 'C4', 'ELT', 'ERT ', 'ERI', 'ELA', 'F4', 'EMGr']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C3', 'C4', 'ERB', 'ERI', 'EMGr', 'O1']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C3', 'C4', 'O2', 'ERI', 'ELC', 'EMGr', 'O1', 'ERB']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERT ', 'C3', 'C4', 'ERI']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ELI', 'M1', 'C4', 'ERT ', 'ERI', 'ELA', 'F4']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C4', 'O1']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ', 'EOGl', 'C4', 'O1']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'F3', 'O1', 'M2', 'F4', 'EMGc', 'ERT ', 'C3', 'ERI']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'F3', 'O1', 'M2', 'EMGc', 'ERT ', 'ERI', 'ELA', 'F4']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERE', 'F3', 'M2', 'F4', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'F3', 'M2', 'F4', 'EMGc', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'F3', 'M2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERE', 'F3', 'F4', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ELB', 'ELE', 'ERE', 'F3', 'F4', 'EMGc', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERE', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ELE', 'F4', 'EMGc', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ELB', 'ELE', 'ERE', 'EMGc', 'ELI', 'ERT ']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ERE', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELE', 'ERC', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGl', 'EMGl', 'F3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ELB', 'ERE', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc', 'EOGl', 'M1', 'C3', 'O2', 'ELA', 'ERC', 'EMGl', 'ELI', 'ERI', 'ELC', 'ERB', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc', 'EOGl', 'M1', 'F4', 'O2', 'ELA', 'ERC', 'C4', 'ELI', 'ERI', 'ERB', 'O1', 'ELC', 'C3']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc', 'ERB', 'EOGl', 'M1', 'O2', 'ERC', 'EMGl', 'C3', 'ELA', 'ERI', 'ELC', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 1, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ERB', 'EOGl', 'O2', 'ERC', 'EMGl', 'ELA', 'ERI', 'C4', 'ELC', 'O1', 'C3']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc', 'M1', 'O2', 'ELA', 'ERC', 'EMGl', 'ELI', 'ERI', 'ELC', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc', 'M1', 'F4', 'O2', 'ELA', 'ERC', 'C4', 'ELI', 'ERI', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc', 'ERB', 'M1', 'O2', 'ERC', 'EMGl', 'ELA', 'ERI', 'ELC', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 2, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ERB', 'O2', 'ERC', 'EMGl', 'ELA', 'C4', 'ELC', 'O1']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'EOGr', 'F3', 'M2', 'F4', 'C4', 'EMGc', 'M1', 'O2', 'ELA', 'ERC', 'EMGl', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERT ', 'ERE', 'F3', 'M2', 'EMGc', 'M1', 'F4', 'O2', 'ELA', 'ERC', 'C4', 'ELI']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'F3', 'M2', 'C4', 'EMGc', 'O2', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 3, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELI', 'ERT ', 'ERE', 'EOGr', 'M1', 'F3', 'M2', 'F4', 'EMGc', 'ERB', 'O2', 'ERC', 'C4']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'F3', 'M2', 'C4', 'ERC']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'F3', 'M2', 'EMGc', 'C4']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELE', 'ELI', 'ERE', 'F3', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 4, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ELI', 'ERE', 'F3', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 5, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'M2']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ELE', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc'], ['ELB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'EMGl', 'M1', 'F3', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGl', 'M1', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELE', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'O1', 'M2', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERT ', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.0, 6, [['ELB', 'ELE', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'EMGc'], ['ELB', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERE', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELB', 'ERE', 'M2', 'EMGc']], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_10', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'ERA', 'ERB', 'ERC', 'EOGr', 'M2', 'O1', 'EMGc', 'ELI', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERI', 'EOGl', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4', 'ELC', 'ERB', 'ERC', 'M2', 'ERI', 'EOGr', 'O1', 'EMGc', 'ELI', 'M1', 'EOGl']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'ERA', 'ERB', 'ERC', 'M2', 'ELE', 'EOGr', 'EOGl', 'F4', 'EMGc', 'ERI', 'O1', 'O2', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'ERA', 'ERB', 'ERC', 'M2', 'ELE', 'EOGr', 'EOGl', 'F4', 'EMGc', 'ERI', 'O1', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELB'], [], [], [], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'C3', 'F4', 'ERA', 'ERC', 'EOGr', 'M2', 'O1', 'EMGc', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERI', 'EOGl', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'C3', 'F4', 'ERC', 'M2', 'ERI', 'O1', 'EMGc', 'EOGl']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3', 'ERA', 'ERC', 'M2', 'ELE', 'EOGl', 'ERI', 'O1', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGr', 'M1', 'C3', 'ERC', 'M2', 'ELE', 'EOGl', 'EMGc', 'ERI', 'O1', 'ELC']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELB'], [], [], [], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'ERI', 'C3', 'F4', 'ERA', 'EOGr', 'M2', 'O1']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERI', 'EOGl', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ERA', 'C3', 'F4', 'M2', 'ERI', 'O1']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ELT', 'M1', 'C3', 'ERA', 'M2', 'ERI']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'M1', 'C3', 'M2', 'ERI']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELB'], [], [], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ERI', 'F4', 'ERA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERI', 'EOGl', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'ERA', 'F4', 'M2', 'ERI']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'M1', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'M1', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELB'], [], [], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'F4', 'ERA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERI', 'EOGl', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'F4', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'M1', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'M2']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELB'], [], [], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERI', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'EMGl', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERA', 'ERB', 'ERC', 'C3', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ERI', 'EOGl', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ELI', 'ERT ', 'EMGl', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ERC', 'EOGr', 'EOGl', 'M1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERI', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELB'], [], [], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELT', 'ELI', 'ERB', 'ERI', 'EMGl', 'O2', 'ELE', 'ERE', 'EOGr', 'EMGr', 'M1', 'ERC', 'F3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ERC', 'EOGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERI', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc', 'ELT', 'ERB', 'EMGl', 'ELE', 'ERE', 'EOGr', 'EMGr', 'M1', 'ERC', 'C3', 'F3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc', 'ELT', 'ERE', 'ERI', 'EMGl', 'O2', 'ELE', 'ERB', 'EOGr', 'EMGr', 'M1', 'ERC', 'F3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc', 'ELT', 'ERE', 'F4', 'O2', 'ELE', 'ELI', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'ERC', 'C3']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 1, [['ELA'], ['ELA', 'ELB'], [], ['ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 1, [['ELB'], [], [], ['ELA'], [], ['ELB']], ['ELB', 'ELA']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'O1', 'M2', 'F4', 'EMGc', 'ELT', 'ERB', 'EMGl', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ERC', 'EOGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERI', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'F4', 'EMGl', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'O1', 'F4', 'EMGc', 'ELT', 'ERI', 'EMGl', 'O2', 'ELE', 'ERB', 'ERC']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'EMGc', 'ELT', 'O2', 'ELE', 'ERB', 'EMGr']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA', 'ELB'], [], ['ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 2, [['ELB'], [], [], ['ELA'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERA', 'M2', 'ELT', 'ERB', 'ELE']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ERC', 'EOGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERI', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ERA', 'ELT', 'O2', 'ERB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ERA', 'ELT', 'O2', 'ERB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 3, [['ELA'], ['ELA', 'ELB'], [], ['ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 3, [['ELB'], [], [], ['ELA'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELT', 'ERB', 'ELE']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ERC', 'EOGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERI', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ELT']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELT']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 4, [['ELA'], ['ELA', 'ELB'], [], ['ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 4, [['ELB'], [], [], ['ELA'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ERC', 'EOGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERI', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ELT']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB', 'ELT']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 5, [['ELA'], ['ELA', 'ELB'], [], ['ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 5, [['ELB'], [], [], ['ELA'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERI', 'EOGl', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELA', 'ELB', 'ERC', 'EOGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERB', 'ERI', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EOGl', 'C3', 'O1', 'M2', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGl', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'F3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERI', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGl', 'EMGl', 'O1', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EMGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'EOGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'EMGr', 'M1', 'C3', 'O1', 'M2']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 1.5, 6, [['ELA'], ['ELA', 'ELB'], [], ['ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 1.5, 6, [['ELB'], [], [], ['ELA'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_30', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'ERB', 'ERI', 'EMGl', 'O2', 'ELE', 'EOGl', 'EMGr', 'ERA', 'EOGr', 'ERE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EMGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ERB', 'EOGl', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4', 'ERC', 'EMGl', 'ERB', 'EOGl', 'F3', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'EMGr', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4', 'ELE', 'EMGl', 'ERB', 'EMGr', 'F3', 'ERA', 'ERE', 'EOGr', 'F4', 'O2']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'C3', 'C4']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4', 'ELE', 'EOGl', 'C3', 'ERB', 'EMGr', 'F3', 'ERA', 'F4', 'EOGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 1, [['ELB'], ['ELB'], [], [], ['ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 1, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 1, [[], [], ['ELA'], [], [], []], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4', 'ERB', 'ELE', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EMGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ERB', 'EOGl', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'M2', 'C4', 'ERB', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'EMGr', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ERC', 'M1', 'C3', 'M2', 'C4', 'ELE', 'ERB', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'C3', 'C4']], ['ELA', 'ELB', 'ELI', 'ERC', 'M1', 'M2', 'C4', 'ELE', 'EOGl', 'C3', 'ERB', 'EMGr']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 2, [['ELB'], ['ELB'], [], [], ['ELB'], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 2, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 2, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'M2', 'C4', 'ERB', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EMGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ERB', 'EOGl', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'M2']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'EMGr', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'ELE', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'C3', 'C4']], ['ELA', 'ELB', 'ELI', 'M1', 'C4', 'ELE', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 3, [['ELB'], ['ELB'], [], [], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 3, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 3, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EMGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ERB', 'EOGl', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'EMGr', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'ELE', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'C3', 'C4']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 4, [['ELB'], ['ELB'], [], [], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 4, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 4, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EMGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ERB', 'EOGl', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'EMGr', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'C3', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 5, [['ELB'], ['ELB'], [], [], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 5, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 5, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGl', 'EMGr', 'M1', 'C3', 'M2', 'C4']], ['ELA', 'ELB', 'ELI', 'C3']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERC', 'EMGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ERB', 'EOGl', 'C3', 'M2', 'C4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EMGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERT ', 'EMGr', 'C3', 'O1', 'M2', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERC', 'ERT ', 'M1', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'EOGl', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'EMGr', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT ', 'EMGr', 'C3', 'C4']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.0, 6, [['ELB'], ['ELB'], [], [], ['ELB'], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 2.5, 6, [['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 3.0, 6, [[], [], ['ELA'], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4', 'ELE', 'ERA', 'F3', 'ERI', 'EMGr', 'F4', 'ELT', 'EOGr', 'EOGl', 'ERE', 'O2']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'M1', 'C3']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2', 'ELE', 'M1', 'C3', 'EMGr', 'EOGl', 'O1', 'EOGr']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'M2', 'O2']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4', 'ELE', 'O1', 'ERI', 'EOGl', 'EMGr', 'ELT', 'EOGr', 'O2']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4', 'ELE', 'ERA', 'F3', 'ERI', 'EOGl', 'EMGr', 'ELT', 'EOGr', 'O2']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 1, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 1, [['ELA', 'ELB'], ['ELA'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], [], [], ['ELA'], [], []], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4', 'ELE', 'ERA', 'ELT', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'M1', 'C3']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'M2', 'ELE', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'M2', 'O2']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'M2', 'C4', 'ELE', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'M1', 'M2', 'C4', 'ELE', 'ERA', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 2, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 2, [['ELA', 'ELB'], ['ELA'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'M1', 'C3', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'M1', 'C3']], ['ELA', 'ELB', 'ELI', 'M1', 'C3']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'M2', 'O2']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'EOGl']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 3, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 3, [['ELA', 'ELB'], ['ELA'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'C3', 'ELE']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'M1', 'C3']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'M2', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB', 'ERC']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 4, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 4, [['ELA', 'ELB'], ['ELA'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'M1', 'C3']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'M2', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERC']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 5, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 5, [['ELA', 'ELB'], ['ELA'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGr', 'C3', 'F4'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'M1', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'M1', 'C3', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'EMGr', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELI', 'O1', 'M2'], ['ELA', 'ELB', 'ELI', 'ERB', 'EOGr', 'M1', 'C3']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'M1', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'EMGr', 'M1', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'M2', 'O2']], ['ELA', 'ELB', 'ELI', 'ERB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERB', 'ERC', 'ERT ', 'M1', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'M1', 'F3', 'M2', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGr'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGl', 'C3', 'C4'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERB', 'ERC', 'EOGl', 'M2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERB', 'EOGr', 'C3', 'C4', 'O2']], ['ELA', 'ELB', 'ELI']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], ['ELA'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.0, 6, [['ELB'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 2.5, 6, [['ELA', 'ELB'], ['ELA'], [], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], [], [], ['ELA'], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_50', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2', 'ELI', 'ERA', 'ERE', 'EOGr', 'F4', 'O2', 'O1', 'C4', 'EMGc', 'M1', 'ELE', 'EOGl']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERA', 'ERT '], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT '], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'M1']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERA', 'M1']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'ELC', 'ELI', 'ERA', 'ERE', 'EOGr', 'F3', 'F4', 'ERI', 'C4', 'EMGc', 'O2', 'ELE']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'F4', 'O2', 'O1', 'C4', 'EMGc', 'ELE']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], [], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'M1']], ['ELA', 'ELB', 'M1']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'ERA', 'ERE', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERA', 'ERT '], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT '], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'M1']], ['ELA', 'ELB', 'ERT ', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERT ', 'C3', 'ELC', 'ERA', 'ERE', 'F4', 'ERI']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'M1', 'C3', 'ELC', 'ERT ', 'ERE', 'EOGr', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], [], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERT ', 'EMGl', 'ERA', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERA', 'ERT '], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT '], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'M1']], ['ELA', 'ELB', 'ERT ']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERT ', 'ELC', 'ERA', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERC', 'ERI', 'M1', 'C3', 'ELC', 'ERT ', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'M2']], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERA', 'ERT '], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT '], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERT ', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERI', 'M1', 'ERT ', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'M2']], ['ELA', 'ELB', 'ERT ', 'ERA']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERA', 'ERT '], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT '], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERT ', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERT ', 'F4']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'F3', 'C3', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EMGl', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERT ', 'ERE', 'EOGl', 'C3'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EMGl', 'M1', 'M2']], ['ELA', 'ELB', 'ERT ']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'ERT '], ['ELA', 'ELB', 'ERA', 'ERT '], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT '], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERC', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'M1', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERT ', 'ERE'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB', 'ERT ']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'ERI', 'EMGl', 'M1', 'C3'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERI', 'EMGl', 'M1', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], [], ['ELA', 'ELB'], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB', 'M1']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 2.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 3.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'ELC', 'ERA', 'O1', 'M2', 'ERE']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERT '], ['ELA', 'ELB', 'EMGl', 'F4'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'C3', 'F4']], ['ELA', 'ELB', 'F4', 'ERT ', 'EMGl', 'ELC', 'C3']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4', 'ELC', 'ELT', 'ERA', 'ERE', 'ERC', 'F3']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4', 'ERA', 'ERC', 'EMGl', 'ELC', 'ERE', 'F3']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA', 'ELB'], [], [], [], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 1, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'ELC', 'ERA', 'ERE']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERT '], ['ELA', 'ELB', 'EMGl', 'F4'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'C3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'ELC', 'ERE']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'ERI', 'EOGr', 'M1', 'F4', 'ELC', 'ERE']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA', 'ELB'], [], [], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERT '], ['ELA', 'ELB', 'EMGl', 'F4'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'C3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'M1', 'F4', 'ERE']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA', 'ELB'], [], [], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 3, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EMGl', 'F4', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERT '], ['ELA', 'ELB', 'EMGl', 'F4'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'C3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'F4', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA', 'ELB'], [], [], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 4, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'EMGl', 'F4', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERT '], ['ELA', 'ELB', 'EMGl', 'F4'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'C3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'EMGl', 'F4', 'ELC']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA', 'ELB'], [], [], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 5, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERT ', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EMGl', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'ERT ', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'ERT '], ['ELA', 'ELB', 'EMGl', 'F4'], ['ELA', 'ELB'], ['ELA', 'ELB', 'ELC', 'C3', 'F4']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EMGl', 'M1', 'C3', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERT ', 'ERI', 'EOGr', 'M1', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'M1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'F4'], ['ELA', 'ELB', 'ELC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'F4'], ['ELA', 'ELB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'O1', 'F4'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'F4']], ['ELA', 'ELB', 'F4']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA', 'ELB'], [], [], [], ['ELA', 'ELB']], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 2.5, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 3.0, 6, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['EEG_manual_100-25cha_2noise_75', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []]]
print( )

## Accuracies

$$
\text{ } \\
Accuracy = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}} = \frac{TP + TN}{TP + TN + FP + FN}\\
$$

In [14]:
def binary_broken_electrode_accuracy(y_preds, y_labels):
    TP = 0 # True positive
    TN = 0 # True negative
    F = 0 # Incorrect - false
    accuracies = []
    
    for data_idx in range(len(y_preds)):
        if len(y_preds[data_idx]) == len(y_labels[data_idx]) == 0:
            # if both lists are empty. 0 labels and 0 predictions.
            TN += 1
            accuracies.append(1)
            
        elif len(y_preds[data_idx]) != len(y_labels[data_idx]):
            # if the lists are not same lenght. Not same number of predictions as number of labels
            F += 1
            accuracies.append(0)
        else:
            # check if predictions are correct.
            total_correct = 0
            for chn in range(len(y_preds[data_idx])):
                if y_preds[data_idx][chn] in y_labels[data_idx]:
                    total_correct += 1
            if total_correct == len(y_labels[data_idx]):
                TP += 1
                accuracies.append(1)
            else:
                F += 1
                accuracies.append(0)
    return (TP + TN) / (TP + TN + F), accuracies, {'TP':TP, 'TN':TN, 'F':F}

In [15]:
def broken_electrode_accuracy(y_preds, y_labels):
    TP = 0 #true_positive
    FP = 0 #false_positive
    TN = 0 #true_negative no broken channels predicted and none labelled.
    FN = 0 #false_negative
    accuracies = []
    
    for data_idx in range(len(y_preds)):
        # Running through each data point
        
        interval_accuracy = []
        for pred_chn in range(len(y_preds[data_idx])):
            # Go through each channel prediction
            
            if y_preds[data_idx][pred_chn] in y_labels[data_idx]:
                # Correctly predicts a broken channel.
                TP += 1
                interval_accuracy.append('TP')
            else:
                # Predict a broken channel, that was not broken.
                FP += 1
                interval_accuracy.append('FP')
        
        for label_chn in range(len(y_labels[data_idx])):
            if y_labels[data_idx][label_chn] not in y_preds[data_idx]:
                # Failed to predict a broken channel.
                FN += 1
                interval_accuracy.append('FN')
        
        if len(y_labels[data_idx]) == 0 and len(y_preds[data_idx]) == 0:
            # Predict no broken and no broken is correct.
            TN += 1
            interval_accuracy.append('TN')
        accuracies.append(interval_accuracy)
    return (TP + TN) / (TP + FP + FN + TN), accuracies, {'TP':TP, 'TN':TN, 'FP':FP, 'FN':FN}

In [16]:
def both_broken_electrode_accuracy(y_preds, y_labels, binary):
    if binary:
        acc_score, acc, types = binary_broken_electrode_accuracy(y_preds, y_labels)
    else:
        acc_score, acc, types = broken_electrode_accuracy(y_preds, y_labels)
    return acc_score, acc, types

## Similiar to Table (1) Left out

In [17]:
def accuracy_per_combination(outputs, data, binary, parameters_boolean):
    #outputs = predictions.
    acc_scores = []
    
    if parameters_boolean:
        # Making empty list for the data.
        # Measures
        acc_scores_single = {}
        y_pre_M_idx = []
        y_lab_M_idx = []
        y_pre_M_max = []
        y_lab_M_max = []
        
        # Matrices
        y_pre_um = []
        y_lab_um = []
        y_pre_mm = []
        y_lab_mm = []
        
        # Relative procents [1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00]
        y_pre_Rel_1 = []
        y_lab_Rel_1 = []
        y_pre_Rel_1_5 = []
        y_lab_Rel_1_5 = []
        y_pre_Rel_2 = []
        y_lab_Rel_2 = []
        y_pre_Rel_2_5 = []
        y_lab_Rel_2_5 = []
        y_pre_Rel_3 = []
        y_lab_Rel_3 = []
        y_pre_Rel_4 = []
        y_lab_Rel_4 = []
        y_pre_Rel_5 = []
        y_lab_Rel_5 = []
        
        # intervals [1, 2, 3, 4, 5, 6]
        y_pre_1 = []
        y_lab_1 = []
        y_pre_2 = []
        y_lab_2 = []
        y_pre_3 = []
        y_lab_3 = []
        y_pre_4 = []
        y_lab_4 = []
        y_pre_5 = []
        y_lab_5 = []
        y_pre_6 = []
        y_lab_6 = []
        
        
    for start_combination in range(0, 168):
        if parameters_boolean:
            # Calculating an accuracy for all combinations of each of the single parameters.
            
            # Measures
            if outputs[start_combination][2] == "M_idx":
                [y_pre_M_idx.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_idx.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][2] == "M_max":
                [y_pre_M_max.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_max.append(data[j][3]) for j in range(0, len(data))]
            
            # Matrices
            if outputs[start_combination][3] == "um":
                [y_pre_um.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_um.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][3] == "mm":
                [y_pre_mm.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_mm.append(data[j][3]) for j in range(0, len(data))]
                
            # Relative procents
            if outputs[start_combination][4] == 1.00:
                [y_pre_Rel_1.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_1.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 1.5:
                [y_pre_Rel_1_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_1_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.00:
                [y_pre_Rel_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.5:
                [y_pre_Rel_2_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 3.00:
                [y_pre_Rel_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 4.00:
                [y_pre_Rel_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 5.00:
                [y_pre_Rel_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_5.append(data[j][3]) for j in range(0, len(data))]
            
            # intervals
            if outputs[start_combination][5] == 1:
                [y_pre_1.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_1.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 2:
                [y_pre_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 3:
                [y_pre_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 4:
                [y_pre_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 5:
                [y_pre_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 6:
                [y_pre_6.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_6.append(data[j][3]) for j in range(0, len(data))]
        else:
            y_pre_each_combination = [outputs[i][7] for i in range(start_combination, len(outputs), 168)]
            y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
            #print("Combination: " + str(outputs[start_combination][2:6]))
        
            acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
            acc_scores.append(acc_score)
            #print(str(acc_score)[:5])
    
    if parameters_boolean:
        acc_scores_single['M_idx'] = both_broken_electrode_accuracy(y_pre_M_idx, y_lab_M_idx, binary)
        acc_scores_single['M_max'] = both_broken_electrode_accuracy(y_pre_M_max, y_lab_M_max, binary)
        acc_scores_single['um'] = both_broken_electrode_accuracy(y_pre_um, y_lab_um, binary)
        acc_scores_single['mm'] = both_broken_electrode_accuracy(y_pre_mm, y_lab_mm, binary)
        acc_scores_single['Rel_1'] = both_broken_electrode_accuracy(y_pre_Rel_1, y_lab_Rel_1, binary)
        acc_scores_single['Rel_1.5'] = both_broken_electrode_accuracy(y_pre_Rel_1_5, y_lab_Rel_1_5, binary)
        acc_scores_single['Rel_2'] = both_broken_electrode_accuracy(y_pre_Rel_2, y_lab_Rel_2, binary)
        acc_scores_single['Rel_2.5'] = both_broken_electrode_accuracy(y_pre_Rel_2_5, y_lab_Rel_2_5, binary)
        acc_scores_single['Rel_3'] = both_broken_electrode_accuracy(y_pre_Rel_3, y_lab_Rel_3, binary)
        acc_scores_single['Rel_4'] = both_broken_electrode_accuracy(y_pre_Rel_4, y_lab_Rel_4, binary)
        acc_scores_single['Rel_5'] = both_broken_electrode_accuracy(y_pre_Rel_5, y_lab_Rel_5, binary)
        acc_scores_single['Int_1'] = both_broken_electrode_accuracy(y_pre_1, y_lab_1, binary)
        acc_scores_single['Int_2'] = both_broken_electrode_accuracy(y_pre_2, y_lab_2, binary)
        acc_scores_single['Int_3'] = both_broken_electrode_accuracy(y_pre_3, y_lab_3, binary)
        acc_scores_single['Int_4'] = both_broken_electrode_accuracy(y_pre_4, y_lab_4, binary)
        acc_scores_single['Int_5'] = both_broken_electrode_accuracy(y_pre_5, y_lab_5, binary)
        acc_scores_single['Int_6'] = both_broken_electrode_accuracy(y_pre_6, y_lab_6, binary)
       
        return acc_scores_single
    else:
        return outputs[np.argmax(acc_scores)][2:6], max(acc_scores)

In [18]:
dictionary = accuracy_per_combination(sim_training_predictions, sim_training_data, binary=True, parameters_boolean=True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.231
M_max=0.357
um=0.215
mm=0.373
Rel_1=0.088
Rel_1.5=0.542
Rel_2=0.342
Rel_2.5=0.301
Rel_3=0.285
Rel_4=0.261
Rel_5=0.239
Int_1=0.305
Int_2=0.3
Int_3=0.291
Int_4=0.288
Int_5=0.285
Int_6=0.295


In [19]:
dictionary = accuracy_per_combination(sim_training_predictions, sim_training_data, binary=False, parameters_boolean=True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.166
M_max=0.260
um=0.153
mm=0.277
Rel_1=0.119
Rel_1.5=0.575
Rel_2=0.328
Rel_2.5=0.266
Rel_3=0.25
Rel_4=0.222
Rel_5=0.202
Int_1=0.141
Int_2=0.181
Int_3=0.221
Int_4=0.256
Int_5=0.276
Int_6=0.297


Accuracy of combinations

## Similiar to Table 2 Left out

In [20]:
def accuracy_per_combination(outputs, data, binary, parameters_boolean):
    #outputs = predictions.
    acc_scores = []
    types_list = []
    
    if parameters_boolean:
        # Making empty list for the data.
        # Measures
        acc_scores_single = {}
        y_pre_M_idx = []
        y_lab_M_idx = []
        y_pre_M_max = []
        y_lab_M_max = []
        
        # Matrices
        y_pre_mm = []
        y_lab_mm = []
        
        # Relative procents [1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00]
        y_pre_Rel_1_5 = []
        y_lab_Rel_1_5 = []
        y_pre_Rel_2 = []
        y_lab_Rel_2 = []
        y_pre_Rel_2_5 = []
        y_lab_Rel_2_5 = []
        y_pre_Rel_3 = []
        y_lab_Rel_3 = []
        y_pre_Rel_4 = []
        y_lab_Rel_4 = []
        y_pre_Rel_5 = []
        y_lab_Rel_5 = []
        
        # intervals [1, 2, 3, 4, 5, 6]
        y_pre_1 = []
        y_lab_1 = []
        y_pre_2 = []
        y_lab_2 = []
        y_pre_3 = []
        y_lab_3 = []
        y_pre_4 = []
        y_lab_4 = []
        y_pre_5 = []
        y_lab_5 = []
        y_pre_6 = []
        y_lab_6 = []
        
        
    for start_combination in range(0, 168):
        if parameters_boolean:
            # Calculating an accuracy for all combinations of each of the single parameters.
            if outputs[start_combination][3] == "um":
                continue
            elif outputs[start_combination][4] == 1.00:
                continue
            #elif outputs[start_combination][5] == 2:
            #    continue
            
            
            # Measures
            if outputs[start_combination][2] == "M_idx":
                [y_pre_M_idx.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_idx.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][2] == "M_max":
                [y_pre_M_max.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_max.append(data[j][3]) for j in range(0, len(data))]
            
            # Matrices
            if outputs[start_combination][3] == "mm":
                [y_pre_mm.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_mm.append(data[j][3]) for j in range(0, len(data))]
                
            # Relative procents
            if outputs[start_combination][4] == 1.5:
                [y_pre_Rel_1_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_1_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.00:
                [y_pre_Rel_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.5:
                [y_pre_Rel_2_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 3.00:
                [y_pre_Rel_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 4.00:
                [y_pre_Rel_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 5.00:
                [y_pre_Rel_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_5.append(data[j][3]) for j in range(0, len(data))]
            
            # intervals
            if outputs[start_combination][5] == 1:
                [y_pre_1.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_1.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 2:
                [y_pre_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 3:
                [y_pre_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 4:
                [y_pre_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 5:
                [y_pre_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 6:
                [y_pre_6.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_6.append(data[j][3]) for j in range(0, len(data))]
        else:
            y_pre_each_combination = [outputs[i][7] for i in range(start_combination, len(outputs), 168)]
            y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
            #print("Combination: " + str(outputs[start_combination][2:6]))
        
            acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
            acc_scores.append(acc_score)
            types_list.append(types)
            #print(str(acc_score)[:5])
    
    if parameters_boolean:
        acc_scores_single['M_idx'] = both_broken_electrode_accuracy(y_pre_M_idx, y_lab_M_idx, binary)
        acc_scores_single['M_max'] = both_broken_electrode_accuracy(y_pre_M_max, y_lab_M_max, binary)
        acc_scores_single['mm'] = both_broken_electrode_accuracy(y_pre_mm, y_lab_mm, binary)
        acc_scores_single['Rel_1.5'] = both_broken_electrode_accuracy(y_pre_Rel_1_5, y_lab_Rel_1_5, binary)
        acc_scores_single['Rel_2'] = both_broken_electrode_accuracy(y_pre_Rel_2, y_lab_Rel_2, binary)
        acc_scores_single['Rel_2.5'] = both_broken_electrode_accuracy(y_pre_Rel_2_5, y_lab_Rel_2_5, binary)
        acc_scores_single['Rel_3'] = both_broken_electrode_accuracy(y_pre_Rel_3, y_lab_Rel_3, binary)
        acc_scores_single['Rel_4'] = both_broken_electrode_accuracy(y_pre_Rel_4, y_lab_Rel_4, binary)
        acc_scores_single['Rel_5'] = both_broken_electrode_accuracy(y_pre_Rel_5, y_lab_Rel_5, binary)
        acc_scores_single['Int_1'] = both_broken_electrode_accuracy(y_pre_1, y_lab_1, binary)
        acc_scores_single['Int_2'] = both_broken_electrode_accuracy(y_pre_2, y_lab_2, binary)
        acc_scores_single['Int_3'] = both_broken_electrode_accuracy(y_pre_3, y_lab_3, binary)
        acc_scores_single['Int_4'] = both_broken_electrode_accuracy(y_pre_4, y_lab_4, binary)
        acc_scores_single['Int_5'] = both_broken_electrode_accuracy(y_pre_5, y_lab_5, binary)
        acc_scores_single['Int_6'] = both_broken_electrode_accuracy(y_pre_6, y_lab_6, binary)
       
        return acc_scores_single
    else:
        # Extract the 10 combination with highest accuracies.
        arg_sort = np.argsort(acc_scores)
        for idx in arg_sort[-10:]:
            print(str(outputs[idx][2:6]) + " " + str(acc_scores[idx]) + str(types_list[idx]))
        return outputs[np.argmax(acc_scores)][2:6], max(acc_scores)

In [21]:
dictionary = accuracy_per_combination(sim_training_predictions, sim_training_data, binary = True, parameters_boolean = True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.272
M_max=0.546
mm=0.409
Rel_1.5=0.597
Rel_2=0.483
Rel_2.5=0.402
Rel_3=0.370
Rel_4=0.322
Rel_5=0.279
Int_1=0.437
Int_2=0.435
Int_3=0.412
Int_4=0.393
Int_5=0.389
Int_6=0.387


In [22]:
dictionary = accuracy_per_combination(sim_training_predictions, sim_training_data, binary = False, parameters_boolean = True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.232
M_max=0.574
mm=0.403
Rel_1.5=0.622
Rel_2=0.488
Rel_2.5=0.389
Rel_3=0.357
Rel_4=0.302
Rel_5=0.261
Int_1=0.432
Int_2=0.427
Int_3=0.406
Int_4=0.388
Int_5=0.385
Int_6=0.382


## Table 4 Best combinations on simulated training set

In [23]:
accuracy_per_combination(sim_training_predictions, sim_training_data, binary=True, parameters_boolean=False)

['M_max', 'mm', 2.0, 4] 0.65{'TP': 18, 'TN': 8, 'F': 14}
['M_max', 'mm', 1.5, 6] 0.65{'TP': 18, 'TN': 8, 'F': 14}
['M_max', 'mm', 2.5, 1] 0.65{'TP': 18, 'TN': 8, 'F': 14}
['M_max', 'mm', 2.0, 2] 0.65{'TP': 18, 'TN': 8, 'F': 14}
['M_max', 'mm', 2.0, 5] 0.65{'TP': 18, 'TN': 8, 'F': 14}
['M_idx', 'um', 1.5, 1] 0.675{'TP': 19, 'TN': 8, 'F': 13}
['M_max', 'mm', 1.5, 1] 0.675{'TP': 22, 'TN': 5, 'F': 13}
['M_idx', 'mm', 1.5, 1] 0.7{'TP': 20, 'TN': 8, 'F': 12}
['M_max', 'mm', 1.5, 3] 0.7{'TP': 20, 'TN': 8, 'F': 12}
['M_max', 'mm', 1.5, 2] 0.75{'TP': 22, 'TN': 8, 'F': 10}


(['M_max', 'mm', 1.5, 2], 0.75)

In [24]:
accuracy_per_combination(sim_training_predictions, sim_training_data, binary=False, parameters_boolean=False)

['M_max', 'mm', 2.0, 5] 0.6785714285714286{'TP': 30, 'TN': 8, 'FP': 0, 'FN': 18}
['M_max', 'mm', 2.0, 6] 0.6785714285714286{'TP': 30, 'TN': 8, 'FP': 0, 'FN': 18}
['M_max', 'mm', 1.5, 6] 0.6785714285714286{'TP': 30, 'TN': 8, 'FP': 0, 'FN': 18}
['M_max', 'mm', 2.0, 2] 0.6785714285714286{'TP': 30, 'TN': 8, 'FP': 0, 'FN': 18}
['M_max', 'mm', 1.5, 5] 0.6964285714285714{'TP': 31, 'TN': 8, 'FP': 0, 'FN': 17}
['M_max', 'mm', 1.5, 4] 0.6964285714285714{'TP': 31, 'TN': 8, 'FP': 0, 'FN': 17}
['M_max', 'mm', 1.5, 1] 0.711864406779661{'TP': 37, 'TN': 5, 'FP': 6, 'FN': 11}
['M_idx', 'mm', 1.5, 1] 0.7142857142857143{'TP': 32, 'TN': 8, 'FP': 0, 'FN': 16}
['M_max', 'mm', 1.5, 3] 0.7321428571428571{'TP': 33, 'TN': 8, 'FP': 0, 'FN': 15}
['M_max', 'mm', 1.5, 2] 0.7857142857142857{'TP': 36, 'TN': 8, 'FP': 0, 'FN': 12}


(['M_max', 'mm', 1.5, 2], 0.7857142857142857)

## Result of the best combination on simulated test set.

In [25]:
check_file_exists(sim_test_data, True)
type(['mm', 'um']) == list

True

In [26]:
#sim_test_prediction = optimise_set(sim_test_data, True, ['M_max', 'mm', 1.5, 2]) # calculation prediction of the best combination

In [27]:
sim_test_prediction = [['EEG_manual_100-25cha_0noise_100', 10, 'M_max', 'mm', 1.5, 2, [[], [], [], [], ['ELE', 'O1'], []], []], ['EEG_manual_100-25cha_0noise_1000', 10, 'M_max', 'mm', 1.5, 2, [[], [], ['F4'], [], [], ['ERA']], []], ['EEG_manual_100-25cha_1noise_100', 4000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_1000', 4000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_30', 2000, 'M_max', 'mm', 1.5, 2, [['ELA'], [], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_1noise_75', 2000, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_30noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['Combinations\\EEG_manual_100-25cha_noise_75noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_100', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_2noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']]]

In [28]:
def test_accuracies(outputs, data, binary):        
    y_pre_each_combination = [outputs[i][7] for i in range(0, len(outputs))]
    y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
    acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
    return acc_score, types
print(test_accuracies(sim_test_prediction, sim_test_data, binary=True))
print(test_accuracies(sim_test_prediction, sim_test_data, binary=False))

(0.9, {'TP': 7, 'TN': 2, 'F': 1})
(0.9285714285714286, {'TP': 11, 'TN': 2, 'FP': 0, 'FN': 1})
